In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {A
    "pycharm": {
     "name": "#%% md\n"
    }
   },
   "source": [
    "# Figures for QPGA paper\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "is_executing": false,
     "name": "#%%\n"
    }
   },
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "\n",
    "import matplotlib.pyplot as plt\n",
    "import matplotlib as mpl\n",
    "from mpl_toolkits.axes_grid1 import make_axes_locatable\n",
    "from matplotlib.colors import ListedColormap\n",
    "\n",
    "import qutip\n",
    "import h5py\n",
    "\n",
    "from qpga import *\n",
    "from qpga.model import QPGA\n",
    "from qpga.circuits import QFT, QFT_layer_count, cluster_state_generator\n",
    "from qpga.training import *\n",
    "from qpga.fidelity_search import *\n",
    "from qpga.linalg import *\n",
    "from qpga.plotting import *\n",
    "from qpga.state_preparation import *\n",
    "from qpga.callbacks import *\n",
    "from qpga.utils import *\n",
    "\n",
    "from tqdm import tqdm_notebook as tqdm\n",
    "\n",
    "import tensorflow as tf\n",
    "from tensorflow.python.keras.optimizers import Adam\n",
    "from tensorflow.python.keras import backend as K\n",
    "K.set_floatx('float64')\n",
    "\n",
    "%matplotlib inline\n",
    "mpl.rcParams['figure.dpi'] = 300\n",
    "\n",
    "%config InlineBackend.figure_format = 'retina'\n",
    "np.set_printoptions(precision=3, linewidth=300)\n",
    "\n",
    "plt.rc('text', usetex=True)\n",
    "plt.rc('font', family='serif')\n",
    "plt.rc('font', size=12)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Fidelity search plot"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "is_executing": false,
     "name": "#%%\n"
    }
   },
   "outputs": [],
   "source": [
    "num_qubits = np.array([2,3,4,5])\n",
    "num_layers_explicit = np.array([QFT_layer_count(n, nearest_neighbor_only=True, include_reshuffling=True) for n in num_qubits])\n",
    "num_layers_SGD = [5, 7, 14, 33] "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "is_executing": false,
     "name": "#%%\n"
    }
   },
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 1800x1050 with 2 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 998,
       "width": 1750
      },
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "def plot_depth_search(num_qubits, num_layers_explicit, num_layers_SGD, savefig=False):\n",
    "    \n",
    "    i = len(num_layers_SGD)\n",
    "    \n",
    "    fig, ax1 = plt.subplots(figsize=(6,3.5))\n",
    "\n",
    "    ax1.plot(num_qubits, num_layers_explicit, label=\"Explicit decomposition\")\n",
    "    ax1.plot(num_qubits[:i], num_layers_SGD, label=\"SGD approximation ($\\mathcal{F} > 99.9\\%$)\")\n",
    "    ax1.set_xlabel(\"Number of qubits\")\n",
    "    ax1.set_ylabel(\"Circuit depth\")\n",
    "    ax1.xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))\n",
    "    ax1.legend()\n",
    "    \n",
    "    ax2 = ax1.twinx()\n",
    "    \n",
    "    color = 'tab:red'\n",
    "    ax2.set_ylim([0, 6.0])\n",
    "    ax2.set_ylabel(\"Relative compactness\", color=color, rotation=270, labelpad=12.0)\n",
    "    ax2.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(1))\n",
    "    ax2.tick_params(axis='y', labelcolor=color)\n",
    "    compactness = num_layers_explicit[:i] / num_layers_SGD\n",
    "    ax2.plot(num_qubits[:i], compactness, color=color, linestyle=\":\")\n",
    "\n",
    "    fig.tight_layout()\n",
    "    \n",
    "    if savefig:\n",
    "        plt.savefig(\"assets/circuit_depth_v2.pdf\", dpi=300, bbox_inches='tight', pad_inches=0)\n",
    "    else:\n",
    "        plt.show()\n",
    "        \n",
    "plot_depth_search(num_qubits, num_layers_explicit, num_layers_SGD, savefig=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## NOON state preparation training plot"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [],
   "source": [
    "N = 4\n",
    "num_samples = 1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Epoch 1/201\n",
      "1/1 - 15s - loss: 0.9720 - antifidelity: 0.9720\n",
      "Epoch 2/201\n",
      "1/1 - 0s - loss: 0.9573 - antifidelity: 0.9573\n",
      "Epoch 3/201\n",
      "1/1 - 0s - loss: 0.9396 - antifidelity: 0.9396\n",
      "Epoch 4/201\n",
      "1/1 - 0s - loss: 0.9190 - antifidelity: 0.9190\n",
      "Epoch 5/201\n",
      "1/1 - 0s - loss: 0.8957 - antifidelity: 0.8957\n",
      "Epoch 6/201\n",
      "1/1 - 0s - loss: 0.8700 - antifidelity: 0.8700\n",
      "Epoch 7/201\n",
      "1/1 - 0s - loss: 0.8423 - antifidelity: 0.8423\n",
      "Epoch 8/201\n",
      "1/1 - 0s - loss: 0.8128 - antifidelity: 0.8128\n",
      "Epoch 9/201\n",
      "1/1 - 0s - loss: 0.7819 - antifidelity: 0.7819\n",
      "Epoch 10/201\n",
      "1/1 - 0s - loss: 0.7501 - antifidelity: 0.7501\n",
      "Epoch 11/201\n",
      "1/1 - 0s - loss: 0.7175 - antifidelity: 0.7175\n",
      "Epoch 12/201\n",
      "1/1 - 0s - loss: 0.6846 - antifidelity: 0.6846\n",
      "Epoch 13/201\n",
      "1/1 - 0s - loss: 0.6517 - antifidelity: 0.6517\n",
      "Epoch 14/201\n",
      "1/1 - 0s - loss: 0.6192 - antifidelity: 0.6192\n",
      "Epoch 15/201\n",
      "1/1 - 0s - loss: 0.5874 - antifidelity: 0.5874\n",
      "Epoch 16/201\n",
      "1/1 - 0s - loss: 0.5568 - antifidelity: 0.5568\n",
      "Epoch 17/201\n",
      "1/1 - 0s - loss: 0.5276 - antifidelity: 0.5276\n",
      "Epoch 18/201\n",
      "1/1 - 0s - loss: 0.4999 - antifidelity: 0.4999\n",
      "Epoch 19/201\n",
      "1/1 - 0s - loss: 0.4736 - antifidelity: 0.4736\n",
      "Epoch 20/201\n",
      "1/1 - 0s - loss: 0.4485 - antifidelity: 0.4485\n",
      "Epoch 21/201\n",
      "1/1 - 0s - loss: 0.4243 - antifidelity: 0.4243\n",
      "Epoch 22/201\n",
      "1/1 - 0s - loss: 0.4008 - antifidelity: 0.4008\n",
      "Epoch 23/201\n",
      "1/1 - 0s - loss: 0.3779 - antifidelity: 0.3779\n",
      "Epoch 24/201\n",
      "1/1 - 0s - loss: 0.3556 - antifidelity: 0.3556\n",
      "Epoch 25/201\n",
      "1/1 - 0s - loss: 0.3341 - antifidelity: 0.3341\n",
      "Epoch 26/201\n",
      "1/1 - 0s - loss: 0.3136 - antifidelity: 0.3136\n",
      "Epoch 27/201\n",
      "1/1 - 0s - loss: 0.2941 - antifidelity: 0.2941\n",
      "Epoch 28/201\n",
      "1/1 - 0s - loss: 0.2758 - antifidelity: 0.2758\n",
      "Epoch 29/201\n",
      "1/1 - 0s - loss: 0.2588 - antifidelity: 0.2588\n",
      "Epoch 30/201\n",
      "1/1 - 0s - loss: 0.2432 - antifidelity: 0.2432\n",
      "Epoch 31/201\n",
      "1/1 - 0s - loss: 0.2288 - antifidelity: 0.2288\n",
      "Epoch 32/201\n",
      "1/1 - 0s - loss: 0.2155 - antifidelity: 0.2155\n",
      "Epoch 33/201\n",
      "1/1 - 0s - loss: 0.2033 - antifidelity: 0.2033\n",
      "Epoch 34/201\n",
      "1/1 - 0s - loss: 0.1918 - antifidelity: 0.1918\n",
      "Epoch 35/201\n",
      "1/1 - 0s - loss: 0.1810 - antifidelity: 0.1810\n",
      "Epoch 36/201\n",
      "1/1 - 0s - loss: 0.1709 - antifidelity: 0.1709\n",
      "Epoch 37/201\n",
      "1/1 - 0s - loss: 0.1613 - antifidelity: 0.1613\n",
      "Epoch 38/201\n",
      "1/1 - 0s - loss: 0.1522 - antifidelity: 0.1522\n",
      "Epoch 39/201\n",
      "1/1 - 0s - loss: 0.1438 - antifidelity: 0.1438\n",
      "Epoch 40/201\n",
      "1/1 - 0s - loss: 0.1358 - antifidelity: 0.1358\n",
      "Epoch 41/201\n",
      "1/1 - 0s - loss: 0.1282 - antifidelity: 0.1282\n",
      "Epoch 42/201\n",
      "1/1 - 0s - loss: 0.1210 - antifidelity: 0.1210\n",
      "Epoch 43/201\n",
      "1/1 - 0s - loss: 0.1141 - antifidelity: 0.1141\n",
      "Epoch 44/201\n",
      "1/1 - 0s - loss: 0.1074 - antifidelity: 0.1074\n",
      "Epoch 45/201\n",
      "1/1 - 0s - loss: 0.1008 - antifidelity: 0.1008\n",
      "Epoch 46/201\n",
      "1/1 - 0s - loss: 0.0944 - antifidelity: 0.0944\n",
      "Epoch 47/201\n",
      "1/1 - 0s - loss: 0.0881 - antifidelity: 0.0881\n",
      "Epoch 48/201\n",
      "1/1 - 0s - loss: 0.0820 - antifidelity: 0.0820\n",
      "Epoch 49/201\n",
      "1/1 - 0s - loss: 0.0761 - antifidelity: 0.0761\n",
      "Epoch 50/201\n",
      "1/1 - 0s - loss: 0.0704 - antifidelity: 0.0704\n",
      "Epoch 51/201\n",
      "1/1 - 0s - loss: 0.0650 - antifidelity: 0.0650\n",
      "Epoch 52/201\n",
      "1/1 - 0s - loss: 0.0598 - antifidelity: 0.0598\n",
      "Epoch 53/201\n",
      "1/1 - 0s - loss: 0.0548 - antifidelity: 0.0548\n",
      "Epoch 54/201\n",
      "1/1 - 0s - loss: 0.0501 - antifidelity: 0.0501\n",
      "Epoch 55/201\n",
      "1/1 - 0s - loss: 0.0457 - antifidelity: 0.0457\n",
      "Epoch 56/201\n",
      "1/1 - 0s - loss: 0.0415 - antifidelity: 0.0415\n",
      "Epoch 57/201\n",
      "1/1 - 0s - loss: 0.0377 - antifidelity: 0.0377\n",
      "Epoch 58/201\n",
      "1/1 - 0s - loss: 0.0341 - antifidelity: 0.0341\n",
      "Epoch 59/201\n",
      "1/1 - 0s - loss: 0.0309 - antifidelity: 0.0309\n",
      "Epoch 60/201\n",
      "1/1 - 0s - loss: 0.0280 - antifidelity: 0.0280\n",
      "Epoch 61/201\n",
      "1/1 - 0s - loss: 0.0254 - antifidelity: 0.0254\n",
      "Epoch 62/201\n",
      "1/1 - 0s - loss: 0.0230 - antifidelity: 0.0230\n",
      "Epoch 63/201\n",
      "1/1 - 0s - loss: 0.0208 - antifidelity: 0.0208\n",
      "Epoch 64/201\n",
      "1/1 - 0s - loss: 0.0188 - antifidelity: 0.0188\n",
      "Epoch 65/201\n",
      "1/1 - 0s - loss: 0.0170 - antifidelity: 0.0170\n",
      "Epoch 66/201\n",
      "1/1 - 0s - loss: 0.0154 - antifidelity: 0.0154\n",
      "Epoch 67/201\n",
      "1/1 - 0s - loss: 0.0139 - antifidelity: 0.0139\n",
      "Epoch 68/201\n",
      "1/1 - 0s - loss: 0.0125 - antifidelity: 0.0125\n",
      "Epoch 69/201\n",
      "1/1 - 0s - loss: 0.0113 - antifidelity: 0.0113\n",
      "Epoch 70/201\n",
      "1/1 - 0s - loss: 0.0101 - antifidelity: 0.0101\n",
      "Epoch 71/201\n",
      "1/1 - 0s - loss: 0.0091 - antifidelity: 0.0091\n",
      "Epoch 72/201\n",
      "1/1 - 0s - loss: 0.0082 - antifidelity: 0.0082\n",
      "Epoch 73/201\n",
      "1/1 - 0s - loss: 0.0074 - antifidelity: 0.0074\n",
      "Epoch 74/201\n",
      "1/1 - 0s - loss: 0.0066 - antifidelity: 0.0066\n",
      "Epoch 75/201\n",
      "1/1 - 0s - loss: 0.0059 - antifidelity: 0.0059\n",
      "Epoch 76/201\n",
      "1/1 - 0s - loss: 0.0053 - antifidelity: 0.0053\n",
      "Epoch 77/201\n",
      "1/1 - 0s - loss: 0.0048 - antifidelity: 0.0048\n",
      "Epoch 78/201\n",
      "1/1 - 0s - loss: 0.0043 - antifidelity: 0.0043\n",
      "Epoch 79/201\n",
      "1/1 - 0s - loss: 0.0039 - antifidelity: 0.0039\n",
      "Epoch 80/201\n",
      "1/1 - 0s - loss: 0.0035 - antifidelity: 0.0035\n",
      "Epoch 81/201\n",
      "1/1 - 0s - loss: 0.0032 - antifidelity: 0.0032\n",
      "Epoch 82/201\n",
      "1/1 - 0s - loss: 0.0029 - antifidelity: 0.0029\n",
      "Epoch 83/201\n",
      "1/1 - 0s - loss: 0.0027 - antifidelity: 0.0027\n",
      "Epoch 84/201\n",
      "1/1 - 0s - loss: 0.0024 - antifidelity: 0.0024\n",
      "Epoch 85/201\n",
      "1/1 - 0s - loss: 0.0022 - antifidelity: 0.0022\n",
      "Epoch 86/201\n",
      "1/1 - 0s - loss: 0.0020 - antifidelity: 0.0020\n",
      "Epoch 87/201\n",
      "1/1 - 0s - loss: 0.0018 - antifidelity: 0.0018\n",
      "Epoch 88/201\n",
      "1/1 - 0s - loss: 0.0017 - antifidelity: 0.0017\n",
      "Epoch 89/201\n",
      "1/1 - 0s - loss: 0.0016 - antifidelity: 0.0016\n",
      "Epoch 90/201\n",
      "1/1 - 0s - loss: 0.0014 - antifidelity: 0.0014\n",
      "Epoch 91/201\n",
      "1/1 - 0s - loss: 0.0013 - antifidelity: 0.0013\n",
      "Epoch 92/201\n",
      "1/1 - 0s - loss: 0.0012 - antifidelity: 0.0012\n",
      "Epoch 93/201\n",
      "1/1 - 0s - loss: 0.0011 - antifidelity: 0.0011\n",
      "Epoch 94/201\n",
      "1/1 - 0s - loss: 0.0010 - antifidelity: 0.0010\n",
      "Epoch 95/201\n",
      "1/1 - 0s - loss: 9.6135e-04 - antifidelity: 9.6135e-04\n",
      "Epoch 96/201\n",
      "1/1 - 0s - loss: 8.8842e-04 - antifidelity: 8.8842e-04\n",
      "Epoch 97/201\n",
      "1/1 - 0s - loss: 8.1909e-04 - antifidelity: 8.1909e-04\n",
      "Epoch 98/201\n",
      "1/1 - 0s - loss: 7.5318e-04 - antifidelity: 7.5318e-04\n",
      "Epoch 99/201\n",
      "1/1 - 0s - loss: 6.9084e-04 - antifidelity: 6.9084e-04\n",
      "Epoch 100/201\n",
      "1/1 - 0s - loss: 6.3226e-04 - antifidelity: 6.3226e-04\n",
      "Epoch 101/201\n",
      "1/1 - 0s - loss: 5.7755e-04 - antifidelity: 5.7755e-04\n",
      "Epoch 102/201\n",
      "1/1 - 0s - loss: 5.2680e-04 - antifidelity: 5.2680e-04\n",
      "Epoch 103/201\n",
      "1/1 - 0s - loss: 4.8006e-04 - antifidelity: 4.8006e-04\n",
      "Epoch 104/201\n",
      "\n",
      "Epoch 00104: ReduceLROnPlateau reducing learning rate to 0.0005.\n",
      "1/1 - 3s - loss: 4.3734e-04 - antifidelity: 4.3734e-04\n",
      "Epoch 105/201\n",
      "1/1 - 0s - loss: 3.9859e-04 - antifidelity: 3.9859e-04\n",
      "Epoch 106/201\n",
      "1/1 - 0s - loss: 3.8951e-04 - antifidelity: 3.8951e-04\n",
      "Epoch 107/201\n",
      "\n",
      "Epoch 00107: ReduceLROnPlateau reducing learning rate to 0.000125.\n",
      "1/1 - 0s - loss: 3.8064e-04 - antifidelity: 3.8064e-04\n",
      "Epoch 108/201\n",
      "1/1 - 0s - loss: 3.7197e-04 - antifidelity: 3.7197e-04\n",
      "Epoch 109/201\n",
      "\n",
      "Epoch 00109: ReduceLROnPlateau reducing learning rate to 3.125e-05.\n",
      "1/1 - 0s - loss: 3.6983e-04 - antifidelity: 3.6983e-04\n",
      "Epoch 110/201\n",
      "1/1 - 0s - loss: 3.6770e-04 - antifidelity: 3.6770e-04\n",
      "Epoch 111/201\n",
      "\n",
      "Epoch 00111: ReduceLROnPlateau reducing learning rate to 7.8125e-06.\n",
      "1/1 - 0s - loss: 3.6717e-04 - antifidelity: 3.6717e-04\n",
      "Epoch 112/201\n",
      "1/1 - 0s - loss: 3.6664e-04 - antifidelity: 3.6664e-04\n",
      "Epoch 113/201\n",
      "\n",
      "Epoch 00113: ReduceLROnPlateau reducing learning rate to 1.953125e-06.\n",
      "1/1 - 0s - loss: 3.6651e-04 - antifidelity: 3.6651e-04\n",
      "Epoch 114/201\n",
      "1/1 - 0s - loss: 3.6638e-04 - antifidelity: 3.6638e-04\n",
      "Epoch 115/201\n",
      "\n",
      "Epoch 00115: ReduceLROnPlateau reducing learning rate to 1e-06.\n",
      "1/1 - 0s - loss: 3.6634e-04 - antifidelity: 3.6634e-04\n",
      "Epoch 116/201\n",
      "1/1 - 0s - loss: 3.6631e-04 - antifidelity: 3.6631e-04\n",
      "Epoch 117/201\n",
      "1/1 - 0s - loss: 3.6629e-04 - antifidelity: 3.6629e-04\n",
      "Epoch 118/201\n",
      "1/1 - 0s - loss: 3.6628e-04 - antifidelity: 3.6628e-04\n",
      "Epoch 119/201\n",
      "1/1 - 0s - loss: 3.6626e-04 - antifidelity: 3.6626e-04\n",
      "Epoch 120/201\n",
      "1/1 - 0s - loss: 3.6624e-04 - antifidelity: 3.6624e-04\n",
      "Epoch 121/201\n",
      "1/1 - 0s - loss: 3.6623e-04 - antifidelity: 3.6623e-04\n",
      "Epoch 122/201\n",
      "1/1 - 0s - loss: 3.6621e-04 - antifidelity: 3.6621e-04\n",
      "Epoch 123/201\n",
      "1/1 - 0s - loss: 3.6619e-04 - antifidelity: 3.6619e-04\n",
      "Epoch 124/201\n",
      "1/1 - 0s - loss: 3.6617e-04 - antifidelity: 3.6617e-04\n",
      "Epoch 125/201\n",
      "1/1 - 0s - loss: 3.6616e-04 - antifidelity: 3.6616e-04\n",
      "Epoch 126/201\n",
      "1/1 - 0s - loss: 3.6614e-04 - antifidelity: 3.6614e-04\n",
      "Epoch 127/201\n",
      "1/1 - 0s - loss: 3.6612e-04 - antifidelity: 3.6612e-04\n",
      "Epoch 128/201\n",
      "1/1 - 0s - loss: 3.6610e-04 - antifidelity: 3.6610e-04\n",
      "Epoch 129/201\n",
      "1/1 - 0s - loss: 3.6609e-04 - antifidelity: 3.6609e-04\n",
      "Epoch 130/201\n",
      "1/1 - 0s - loss: 3.6607e-04 - antifidelity: 3.6607e-04\n",
      "Epoch 131/201\n",
      "1/1 - 0s - loss: 3.6605e-04 - antifidelity: 3.6605e-04\n",
      "Epoch 132/201\n",
      "1/1 - 0s - loss: 3.6603e-04 - antifidelity: 3.6603e-04\n",
      "Epoch 133/201\n",
      "1/1 - 0s - loss: 3.6602e-04 - antifidelity: 3.6602e-04\n",
      "Epoch 134/201\n",
      "1/1 - 0s - loss: 3.6600e-04 - antifidelity: 3.6600e-04\n",
      "Epoch 135/201\n",
      "1/1 - 0s - loss: 3.6598e-04 - antifidelity: 3.6598e-04\n",
      "Epoch 136/201\n",
      "1/1 - 0s - loss: 3.6596e-04 - antifidelity: 3.6596e-04\n",
      "Epoch 137/201\n",
      "1/1 - 0s - loss: 3.6594e-04 - antifidelity: 3.6594e-04\n",
      "Epoch 138/201\n",
      "1/1 - 0s - loss: 3.6593e-04 - antifidelity: 3.6593e-04\n",
      "Epoch 139/201\n",
      "1/1 - 0s - loss: 3.6591e-04 - antifidelity: 3.6591e-04\n",
      "Epoch 140/201\n",
      "1/1 - 0s - loss: 3.6589e-04 - antifidelity: 3.6589e-04\n",
      "Epoch 141/201\n",
      "1/1 - 0s - loss: 3.6587e-04 - antifidelity: 3.6587e-04\n",
      "Epoch 142/201\n",
      "1/1 - 0s - loss: 3.6585e-04 - antifidelity: 3.6585e-04\n",
      "Epoch 143/201\n",
      "1/1 - 0s - loss: 3.6583e-04 - antifidelity: 3.6583e-04\n",
      "Epoch 144/201\n",
      "1/1 - 0s - loss: 3.6581e-04 - antifidelity: 3.6581e-04\n",
      "Epoch 145/201\n",
      "1/1 - 0s - loss: 3.6580e-04 - antifidelity: 3.6580e-04\n",
      "Epoch 146/201\n",
      "1/1 - 0s - loss: 3.6578e-04 - antifidelity: 3.6578e-04\n",
      "Epoch 147/201\n",
      "1/1 - 0s - loss: 3.6576e-04 - antifidelity: 3.6576e-04\n",
      "Epoch 148/201\n",
      "1/1 - 0s - loss: 3.6574e-04 - antifidelity: 3.6574e-04\n",
      "Epoch 149/201\n",
      "1/1 - 0s - loss: 3.6572e-04 - antifidelity: 3.6572e-04\n",
      "Epoch 150/201\n",
      "1/1 - 0s - loss: 3.6570e-04 - antifidelity: 3.6570e-04\n",
      "Epoch 151/201\n",
      "1/1 - 0s - loss: 3.6568e-04 - antifidelity: 3.6568e-04\n",
      "Epoch 152/201\n",
      "1/1 - 0s - loss: 3.6566e-04 - antifidelity: 3.6566e-04\n",
      "Epoch 153/201\n",
      "1/1 - 0s - loss: 3.6565e-04 - antifidelity: 3.6565e-04\n",
      "Epoch 154/201\n",
      "1/1 - 0s - loss: 3.6563e-04 - antifidelity: 3.6563e-04\n",
      "Epoch 155/201\n",
      "1/1 - 0s - loss: 3.6561e-04 - antifidelity: 3.6561e-04\n",
      "Epoch 156/201\n",
      "1/1 - 0s - loss: 3.6559e-04 - antifidelity: 3.6559e-04\n",
      "Epoch 157/201\n",
      "1/1 - 0s - loss: 3.6557e-04 - antifidelity: 3.6557e-04\n",
      "Epoch 158/201\n",
      "1/1 - 0s - loss: 3.6555e-04 - antifidelity: 3.6555e-04\n",
      "Epoch 159/201\n",
      "1/1 - 0s - loss: 3.6553e-04 - antifidelity: 3.6553e-04\n",
      "Epoch 160/201\n",
      "1/1 - 0s - loss: 3.6551e-04 - antifidelity: 3.6551e-04\n",
      "Epoch 161/201\n",
      "1/1 - 0s - loss: 3.6549e-04 - antifidelity: 3.6549e-04\n",
      "Epoch 162/201\n",
      "1/1 - 0s - loss: 3.6547e-04 - antifidelity: 3.6547e-04\n",
      "Epoch 163/201\n",
      "1/1 - 0s - loss: 3.6545e-04 - antifidelity: 3.6545e-04\n",
      "Epoch 164/201\n",
      "1/1 - 0s - loss: 3.6543e-04 - antifidelity: 3.6543e-04\n",
      "Epoch 165/201\n",
      "1/1 - 0s - loss: 3.6541e-04 - antifidelity: 3.6541e-04\n",
      "Epoch 166/201\n",
      "1/1 - 0s - loss: 3.6539e-04 - antifidelity: 3.6539e-04\n",
      "Epoch 167/201\n",
      "1/1 - 0s - loss: 3.6537e-04 - antifidelity: 3.6537e-04\n",
      "Epoch 168/201\n",
      "1/1 - 0s - loss: 3.6535e-04 - antifidelity: 3.6535e-04\n",
      "Epoch 169/201\n",
      "1/1 - 0s - loss: 3.6533e-04 - antifidelity: 3.6533e-04\n",
      "Epoch 170/201\n",
      "1/1 - 0s - loss: 3.6531e-04 - antifidelity: 3.6531e-04\n",
      "Epoch 171/201\n",
      "1/1 - 0s - loss: 3.6529e-04 - antifidelity: 3.6529e-04\n",
      "Epoch 172/201\n",
      "1/1 - 0s - loss: 3.6527e-04 - antifidelity: 3.6527e-04\n",
      "Epoch 173/201\n",
      "1/1 - 0s - loss: 3.6525e-04 - antifidelity: 3.6525e-04\n",
      "Epoch 174/201\n",
      "1/1 - 0s - loss: 3.6523e-04 - antifidelity: 3.6523e-04\n",
      "Epoch 175/201\n",
      "1/1 - 0s - loss: 3.6521e-04 - antifidelity: 3.6521e-04\n",
      "Epoch 176/201\n",
      "1/1 - 0s - loss: 3.6519e-04 - antifidelity: 3.6519e-04\n",
      "Epoch 177/201\n",
      "1/1 - 0s - loss: 3.6517e-04 - antifidelity: 3.6517e-04\n",
      "Epoch 178/201\n",
      "1/1 - 0s - loss: 3.6515e-04 - antifidelity: 3.6515e-04\n",
      "Epoch 179/201\n",
      "1/1 - 0s - loss: 3.6513e-04 - antifidelity: 3.6513e-04\n",
      "Epoch 180/201\n",
      "1/1 - 0s - loss: 3.6511e-04 - antifidelity: 3.6511e-04\n",
      "Epoch 181/201\n",
      "1/1 - 0s - loss: 3.6509e-04 - antifidelity: 3.6509e-04\n",
      "Epoch 182/201\n",
      "1/1 - 0s - loss: 3.6506e-04 - antifidelity: 3.6506e-04\n",
      "Epoch 183/201\n",
      "1/1 - 0s - loss: 3.6504e-04 - antifidelity: 3.6504e-04\n",
      "Epoch 184/201\n",
      "1/1 - 0s - loss: 3.6502e-04 - antifidelity: 3.6502e-04\n",
      "Epoch 185/201\n",
      "1/1 - 0s - loss: 3.6500e-04 - antifidelity: 3.6500e-04\n",
      "Epoch 186/201\n",
      "1/1 - 0s - loss: 3.6498e-04 - antifidelity: 3.6498e-04\n",
      "Epoch 187/201\n",
      "1/1 - 0s - loss: 3.6496e-04 - antifidelity: 3.6496e-04\n",
      "Epoch 188/201\n",
      "1/1 - 0s - loss: 3.6494e-04 - antifidelity: 3.6494e-04\n",
      "Epoch 189/201\n",
      "1/1 - 0s - loss: 3.6492e-04 - antifidelity: 3.6492e-04\n",
      "Epoch 190/201\n",
      "1/1 - 0s - loss: 3.6489e-04 - antifidelity: 3.6489e-04\n",
      "Epoch 191/201\n",
      "1/1 - 0s - loss: 3.6487e-04 - antifidelity: 3.6487e-04\n",
      "Epoch 192/201\n",
      "1/1 - 0s - loss: 3.6485e-04 - antifidelity: 3.6485e-04\n",
      "Epoch 193/201\n",
      "1/1 - 0s - loss: 3.6483e-04 - antifidelity: 3.6483e-04\n",
      "Epoch 194/201\n",
      "1/1 - 0s - loss: 3.6481e-04 - antifidelity: 3.6481e-04\n",
      "Epoch 195/201\n",
      "1/1 - 0s - loss: 3.6479e-04 - antifidelity: 3.6479e-04\n",
      "Epoch 196/201\n",
      "1/1 - 0s - loss: 3.6477e-04 - antifidelity: 3.6477e-04\n",
      "Epoch 197/201\n",
      "1/1 - 0s - loss: 3.6474e-04 - antifidelity: 3.6474e-04\n",
      "Epoch 198/201\n",
      "1/1 - 0s - loss: 3.6472e-04 - antifidelity: 3.6472e-04\n",
      "Epoch 199/201\n",
      "1/1 - 0s - loss: 3.6470e-04 - antifidelity: 3.6470e-04\n",
      "Epoch 200/201\n",
      "1/1 - 0s - loss: 3.6468e-04 - antifidelity: 3.6468e-04\n",
      "Epoch 201/201\n",
      "1/1 - 0s - loss: 3.6466e-04 - antifidelity: 3.6466e-04\n",
      "Writing data to ./state_history_4_qubits_2019.08.19.17.51.17.h5...\n"
     ]
    }
   ],
   "source": [
    "in_data = np_to_k_complex(np.array([zero_state(N)] * num_samples))\n",
    "out_data = np_to_k_complex(np.array([noon_state(N)] * num_samples))\n",
    "\n",
    "model = QPGA(N, 20).as_sequential()\n",
    "model.compile(optimizer=Adam(lr=0.002), \n",
    "              loss=antifidelity, \n",
    "              metrics=[antifidelity])\n",
    "\n",
    "callback = StatePreparationHistoryCallback(num_qubits=N, input_state = in_data[0:1], target_state = out_data[0:1])\n",
    "reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(monitor = 'loss', \n",
    "                                                       factor = 0.25,\n",
    "                                                       patience = 2,\n",
    "                                                       verbose=1,\n",
    "                                                       min_lr=1e-6)\n",
    "\n",
    "history = model.fit(in_data, out_data, epochs=201, \n",
    "                    callbacks=[callback, reduce_lr_callback], verbose = 2)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 2400x1500 with 4 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 1433,
       "width": 2334
      },
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "def plot_state_prep_training(filepath, max_epoch=None, savefig=False):\n",
    "    \n",
    "    f = h5py.File(filepath, 'r')\n",
    "    \n",
    "    input_state = np.array(f['input_state'])\n",
    "    target_state = np.array(f['target_state'])\n",
    "    output_states = np.array(f['output_states'])\n",
    "    fidelities = np.array(f['fidelities'])\n",
    "    \n",
    "    output_states = k_to_np_complex(np.array(output_states)[:,0])\n",
    "    \n",
    "    if max_epoch is not None:\n",
    "        output_states = output_states[:max_epoch]\n",
    "        fidelities = fidelities[:max_epoch]\n",
    "    \n",
    "    kets = computational_basis_labels(N, include_bras=False)\n",
    "    \n",
    "    # Make figure and axis layout\n",
    "    plt.rc('text', usetex=True)\n",
    "    plt.rc('font', family='serif')\n",
    "    plt.rc('font', size=14)\n",
    "\n",
    "    fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(8,5), gridspec_kw={'height_ratios': [2,1]}, sharex=True)\n",
    "\n",
    "    # Plot state over training\n",
    "    cmap='viridis'\n",
    "    ax1.imshow(np.abs(np.transpose(output_states)), aspect='auto', cmap=cmap)\n",
    "    ax1.set_xticks([])\n",
    "    ax1.yaxis.set_major_locator(plt.IndexLocator(base=1, offset=0.35))\n",
    "    ax1.yaxis.set_ticks_position('none')\n",
    "    ax1.set_yticklabels(kets)\n",
    "    ax1.tick_params(axis = 'y', labelsize = 10, pad = 0)\n",
    "    \n",
    "    # Add a colorbar\n",
    "    divider = make_axes_locatable(ax1)\n",
    "    cax = divider.append_axes('right', size = '2%', pad = '1%')\n",
    "    cbar = mpl.colorbar.ColorbarBase(cax, cmap = cmap,\n",
    "                                     norm = mpl.colors.Normalize(0, 1/np.sqrt(2)),\n",
    "                                     ticks = [0, np.sqrt(1/2)])\n",
    "    cax.get_yaxis().labelpad=-4\n",
    "    cax.set_ylabel(\"Component amplitude\", rotation=270)\n",
    "    cax.set_yticklabels([\"$0$\", \"$\\\\frac{1}{\\sqrt{2}}$\"])\n",
    "#     cax.text(0.5, 0.0, '$0$', transform = cax.transAxes, va = 'top', ha = 'center')\n",
    "#     cax.text(0.5, 1.0, '$1/\\sqrt{2}$', transform = cax.transAxes, va = 'bottom', ha = 'center')\n",
    "\n",
    "    # Plot antifidelity\n",
    "    loss_plot(fidelities, x_units='iterations', fig=fig, ax=ax2, ylabel=\"$\\mathcal{F} = | \\langle \\\\tilde{\\psi} | \\psi \\\\rangle |^2$\", ylabel_pos='right')\n",
    "    # Add a dummy axis to keep alignment\n",
    "    divider = make_axes_locatable(ax2)\n",
    "    dummyax = divider.append_axes('right', size = '2%', pad = '1%')\n",
    "    dummyax.set_visible(False)\n",
    "    \n",
    "    fig.tight_layout(h_pad=0)\n",
    "    \n",
    "    # Save frame\n",
    "    if savefig:\n",
    "        plt.savefig(\"assets/noon_prep_v2.pdf\", dpi=300, bbox_inches='tight', pad_inches=0.025)\n",
    "    else:\n",
    "        plt.show()\n",
    "\n",
    "plot_state_prep_training(\"./logs/state_history_4_qubits_2019.08.19.17.51.17.h5\", max_epoch=101, savefig=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Random state generation figure"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "51646dc4b0d340d396458a5bd131f95b",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "HBox(children=(IntProgress(value=0, max=20), HTML(value='')))"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "Writing data to random_state_preparation_4_qubits.h5...\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "W0913 12:49:46.531899 4442961344 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (14.114006). Check your callbacks.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "W0913 12:51:27.938740 4442961344 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (13.946497). Check your callbacks.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "W0913 12:53:15.356517 4442961344 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (15.443210). Check your callbacks.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "W0913 12:55:10.797096 4442961344 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (17.149635). Check your callbacks.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "W0913 12:57:34.382793 4442961344 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (17.061403). Check your callbacks.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "W0913 12:59:42.175717 4442961344 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (18.516836). Check your callbacks.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "W0913 13:01:52.350552 4442961344 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (20.576682). Check your callbacks.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "W0913 13:04:41.018559 4442961344 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (20.061487). Check your callbacks.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "W0913 13:07:10.217306 4442961344 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (25.278432). Check your callbacks.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Writing data to random_state_preparation_4_qubits.h5...\n",
      "\n"
     ]
    }
   ],
   "source": [
    "N = 4\n",
    "\n",
    "num_states_to_generate = 20\n",
    "\n",
    "target_states = np.array([get_random_state_vector(N) for _ in range(num_states_to_generate)])\n",
    "\n",
    "filename = \"random_state_preparation_{}_qubits.h5\".format(N)\n",
    "\n",
    "for i in tqdm(range(num_states_to_generate)):\n",
    "\n",
    "    in_data = np_to_k_complex(np.array([zero_state(N)] * 1))\n",
    "    out_data = np_to_k_complex(np.array([target_states[i]] * 1))\n",
    "\n",
    "    model = QPGA(N, 20).as_sequential()\n",
    "    model.compile(optimizer=Adam(lr=0.01), \n",
    "                  loss=antifidelity, \n",
    "                  metrics=[antifidelity])\n",
    "\n",
    "    callback = StatePreparationHistoryCallback(num_qubits = N, \n",
    "                                               input_state = in_data[0:1], \n",
    "                                               target_state = out_data[0:1],\n",
    "                                               filename = filename,\n",
    "                                               groupname = str(i))\n",
    "    \n",
    "    reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(monitor = 'loss', \n",
    "                                                           factor = 0.25,\n",
    "                                                           patience = 2,\n",
    "                                                           verbose=0,\n",
    "                                                           min_lr=1e-6)\n",
    "\n",
    "    history = model.fit(in_data, out_data, epochs=101, \n",
    "                        callbacks=[callback, reduce_lr_callback], verbose = 0)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 51,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Mean final fidelity: 0.9922617312009543\n"
     ]
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 1800x900 with 1 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 883,
       "width": 1623
      },
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "def plot_random_state_prep(filepath, max_epoch=None, log_fidelity=False, savefig=False):\n",
    "    \n",
    "    # Make figure and axis layout\n",
    "    plt.rc('text', usetex=True)\n",
    "    plt.rc('font', family='serif')\n",
    "\n",
    "    fig, ax = plt.subplots(1,1,figsize=(6,3))\n",
    "    \n",
    "    f = h5py.File(filepath, 'r')\n",
    "    \n",
    "    final_fidelities = []\n",
    "    \n",
    "    for group in f.keys():\n",
    "        group = f[group]\n",
    "        \n",
    "        fidelities = np.array(group['fidelities'])\n",
    "\n",
    "        final_fidelities.append(fidelities[-1])\n",
    "        \n",
    "        if max_epoch is not None:\n",
    "            fidelities = fidelities[:max_epoch]\n",
    "\n",
    "        ax.plot(fidelities, linestyle='-', linewidth=0.25)\n",
    "        ax.set_xlim(0, len(fidelities)-1)\n",
    "\n",
    "        \n",
    "    ax.set_ylim(0, 1)\n",
    "    ax.set_ylabel(\"$\\mathcal{F} = | \\langle \\\\tilde{\\psi} | \\psi \\\\rangle |^2$\")\n",
    "\n",
    "    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))\n",
    "    ax.set_xlabel(\"Iteration\")\n",
    "    \n",
    "    plt.text(.99, 0.9, \"$\\overline{\\mathcal{F}} = \"+\"{:.3f}\".format(np.mean(final_fidelities))+\"$\", horizontalalignment='right', verticalalignment='center', transform=ax.transAxes, \n",
    "             fontdict={\"size\":7, \n",
    "#                        \"color\":\"grey\"\n",
    "                      })\n",
    "        \n",
    "    print(\"Mean final fidelity: {}\".format(np.mean(final_fidelities)))\n",
    "    \n",
    "    # Save frame\n",
    "    if savefig:\n",
    "        plt.savefig(\"assets/random_state_prep.pdf\", dpi=300, bbox_inches='tight', pad_inches=0.0)\n",
    "    else:\n",
    "        plt.show()\n",
    "\n",
    "plot_random_state_prep(\"./logs/random_state_preparation_4_qubits.h5\", max_epoch=51, savefig=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## QFT plot"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "is_executing": false,
     "name": "#%%\n"
    }
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "100%|██████████| 1000/1000 [00:01<00:00, 769.62it/s]\n"
     ]
    }
   ],
   "source": [
    "N = 5\n",
    "in_data, out_data = prepare_training_data(QFT, N, 1000)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "is_executing": false,
     "name": "#%%\n"
    }
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Train on 900 samples, validate on 100 samples\n",
      "1000/1000 [==============================] - 71s 71ms/sample - loss: 0.9687 - antifidelity: 0.9687\n",
      "Initial fidelity: [0.9686641327738762, 0.9686641327059634]\n",
      "Epoch 1/200\n",
      "900/900 [==============================] - 203s 226ms/sample - loss: 0.9678 - antifidelity: 0.9678 - val_loss: 0.9685 - val_antifidelity: 0.9685\n",
      "Epoch 2/200\n",
      "900/900 [==============================] - 17s 19ms/sample - loss: 0.9631 - antifidelity: 0.9631 - val_loss: 0.9644 - val_antifidelity: 0.9644\n",
      "Epoch 3/200\n",
      "900/900 [==============================] - 16s 18ms/sample - loss: 0.9551 - antifidelity: 0.9551 - val_loss: 0.9553 - val_antifidelity: 0.9553\n",
      "Epoch 4/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.9410 - antifidelity: 0.9410 - val_loss: 0.9375 - val_antifidelity: 0.9375\n",
      "Epoch 5/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.9194 - antifidelity: 0.9194 - val_loss: 0.9125 - val_antifidelity: 0.9125\n",
      "Epoch 6/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.8918 - antifidelity: 0.8918 - val_loss: 0.8823 - val_antifidelity: 0.8823\n",
      "Epoch 7/200\n",
      "900/900 [==============================] - 15s 17ms/sample - loss: 0.8602 - antifidelity: 0.8602 - val_loss: 0.8486 - val_antifidelity: 0.8486\n",
      "Epoch 8/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.8266 - antifidelity: 0.8266 - val_loss: 0.8132 - val_antifidelity: 0.8132\n",
      "Epoch 9/200\n",
      "900/900 [==============================] - 15s 17ms/sample - loss: 0.7917 - antifidelity: 0.7917 - val_loss: 0.7759 - val_antifidelity: 0.7759\n",
      "Epoch 10/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.7554 - antifidelity: 0.7554 - val_loss: 0.7371 - val_antifidelity: 0.7371\n",
      "Epoch 11/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.7184 - antifidelity: 0.7184 - val_loss: 0.6972 - val_antifidelity: 0.6972\n",
      "Epoch 12/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.6804 - antifidelity: 0.6804 - val_loss: 0.6571 - val_antifidelity: 0.6571\n",
      "Epoch 13/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.6429 - antifidelity: 0.6429 - val_loss: 0.6172 - val_antifidelity: 0.6172\n",
      "Epoch 14/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.6060 - antifidelity: 0.6060 - val_loss: 0.5790 - val_antifidelity: 0.5790\n",
      "Epoch 15/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.5703 - antifidelity: 0.5703 - val_loss: 0.5428 - val_antifidelity: 0.5428\n",
      "Epoch 16/200\n",
      "900/900 [==============================] - 16s 17ms/sample - loss: 0.5362 - antifidelity: 0.5362 - val_loss: 0.5074 - val_antifidelity: 0.5074\n",
      "Epoch 17/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.5039 - antifidelity: 0.5039 - val_loss: 0.4751 - val_antifidelity: 0.4751\n",
      "Epoch 18/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.4737 - antifidelity: 0.4737 - val_loss: 0.4446 - val_antifidelity: 0.4446\n",
      "Epoch 19/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.4453 - antifidelity: 0.4453 - val_loss: 0.4171 - val_antifidelity: 0.4171\n",
      "Epoch 20/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.4189 - antifidelity: 0.4189 - val_loss: 0.3914 - val_antifidelity: 0.3914\n",
      "Epoch 21/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.3941 - antifidelity: 0.3941 - val_loss: 0.3679 - val_antifidelity: 0.3679\n",
      "Epoch 22/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.3708 - antifidelity: 0.3708 - val_loss: 0.3462 - val_antifidelity: 0.3462\n",
      "Epoch 23/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.3490 - antifidelity: 0.3490 - val_loss: 0.3256 - val_antifidelity: 0.3256\n",
      "Epoch 24/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.3284 - antifidelity: 0.3284 - val_loss: 0.3068 - val_antifidelity: 0.3068\n",
      "Epoch 25/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.3089 - antifidelity: 0.3089 - val_loss: 0.2893 - val_antifidelity: 0.2893\n",
      "Epoch 26/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.2906 - antifidelity: 0.2906 - val_loss: 0.2734 - val_antifidelity: 0.2734\n",
      "Epoch 27/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.2737 - antifidelity: 0.2737 - val_loss: 0.2578 - val_antifidelity: 0.2578\n",
      "Epoch 28/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.2575 - antifidelity: 0.2575 - val_loss: 0.2436 - val_antifidelity: 0.2436\n",
      "Epoch 29/200\n",
      "900/900 [==============================] - 15s 17ms/sample - loss: 0.2425 - antifidelity: 0.2425 - val_loss: 0.2299 - val_antifidelity: 0.2299\n",
      "Epoch 30/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.2284 - antifidelity: 0.2284 - val_loss: 0.2173 - val_antifidelity: 0.2173\n",
      "Epoch 31/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.2152 - antifidelity: 0.2152 - val_loss: 0.2053 - val_antifidelity: 0.2053\n",
      "Epoch 32/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.2030 - antifidelity: 0.2030 - val_loss: 0.1941 - val_antifidelity: 0.1941\n",
      "Epoch 33/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.1916 - antifidelity: 0.1916 - val_loss: 0.1833 - val_antifidelity: 0.1833\n",
      "Epoch 34/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.1810 - antifidelity: 0.1810 - val_loss: 0.1733 - val_antifidelity: 0.1733\n",
      "Epoch 35/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.1709 - antifidelity: 0.1709 - val_loss: 0.1642 - val_antifidelity: 0.1642\n",
      "Epoch 36/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.1618 - antifidelity: 0.1618 - val_loss: 0.1557 - val_antifidelity: 0.1557\n",
      "Epoch 37/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.1531 - antifidelity: 0.1531 - val_loss: 0.1476 - val_antifidelity: 0.1476\n",
      "Epoch 38/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.1451 - antifidelity: 0.1451 - val_loss: 0.1398 - val_antifidelity: 0.1398\n",
      "Epoch 39/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.1375 - antifidelity: 0.1375 - val_loss: 0.1327 - val_antifidelity: 0.1327\n",
      "Epoch 40/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.1304 - antifidelity: 0.1304 - val_loss: 0.1259 - val_antifidelity: 0.1259\n",
      "Epoch 41/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.1238 - antifidelity: 0.1238 - val_loss: 0.1197 - val_antifidelity: 0.1197\n",
      "Epoch 42/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.1175 - antifidelity: 0.1175 - val_loss: 0.1137 - val_antifidelity: 0.1137\n",
      "Epoch 43/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.1116 - antifidelity: 0.1116 - val_loss: 0.1081 - val_antifidelity: 0.1081\n",
      "Epoch 44/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.1060 - antifidelity: 0.1060 - val_loss: 0.1027 - val_antifidelity: 0.1027\n",
      "Epoch 45/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.1009 - antifidelity: 0.1009 - val_loss: 0.0978 - val_antifidelity: 0.0978\n",
      "Epoch 46/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0960 - antifidelity: 0.0960 - val_loss: 0.0931 - val_antifidelity: 0.0931\n",
      "Epoch 47/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0914 - antifidelity: 0.0914 - val_loss: 0.0885 - val_antifidelity: 0.0885\n",
      "Epoch 48/200\n",
      "900/900 [==============================] - 15s 17ms/sample - loss: 0.0871 - antifidelity: 0.0871 - val_loss: 0.0844 - val_antifidelity: 0.0844\n",
      "Epoch 49/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0830 - antifidelity: 0.0830 - val_loss: 0.0804 - val_antifidelity: 0.0804\n",
      "Epoch 50/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0791 - antifidelity: 0.0791 - val_loss: 0.0767 - val_antifidelity: 0.0767\n",
      "Epoch 51/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0753 - antifidelity: 0.0753 - val_loss: 0.0731 - val_antifidelity: 0.0731\n",
      "Epoch 52/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0718 - antifidelity: 0.0718 - val_loss: 0.0697 - val_antifidelity: 0.0697\n",
      "Epoch 53/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0685 - antifidelity: 0.0685 - val_loss: 0.0664 - val_antifidelity: 0.0664\n",
      "Epoch 54/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0654 - antifidelity: 0.0654 - val_loss: 0.0633 - val_antifidelity: 0.0633\n",
      "Epoch 55/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0624 - antifidelity: 0.0624 - val_loss: 0.0605 - val_antifidelity: 0.0605\n",
      "Epoch 56/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0595 - antifidelity: 0.0595 - val_loss: 0.0578 - val_antifidelity: 0.0578\n",
      "Epoch 57/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0568 - antifidelity: 0.0568 - val_loss: 0.0552 - val_antifidelity: 0.0552\n",
      "Epoch 58/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0543 - antifidelity: 0.0543 - val_loss: 0.0527 - val_antifidelity: 0.0527\n",
      "Epoch 59/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0518 - antifidelity: 0.0518 - val_loss: 0.0503 - val_antifidelity: 0.0503\n",
      "Epoch 60/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0495 - antifidelity: 0.0495 - val_loss: 0.0480 - val_antifidelity: 0.0480\n",
      "Epoch 61/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0473 - antifidelity: 0.0473 - val_loss: 0.0459 - val_antifidelity: 0.0459\n",
      "Epoch 62/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0452 - antifidelity: 0.0452 - val_loss: 0.0438 - val_antifidelity: 0.0438\n",
      "Epoch 63/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0432 - antifidelity: 0.0432 - val_loss: 0.0419 - val_antifidelity: 0.0419\n",
      "Epoch 64/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0413 - antifidelity: 0.0413 - val_loss: 0.0401 - val_antifidelity: 0.0401\n",
      "Epoch 65/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0395 - antifidelity: 0.0395 - val_loss: 0.0383 - val_antifidelity: 0.0383\n",
      "Epoch 66/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0378 - antifidelity: 0.0378 - val_loss: 0.0367 - val_antifidelity: 0.0367\n",
      "Epoch 67/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0361 - antifidelity: 0.0361 - val_loss: 0.0351 - val_antifidelity: 0.0351\n",
      "Epoch 68/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0346 - antifidelity: 0.0346 - val_loss: 0.0336 - val_antifidelity: 0.0336\n",
      "Epoch 69/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0331 - antifidelity: 0.0331 - val_loss: 0.0321 - val_antifidelity: 0.0321\n",
      "Epoch 70/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0317 - antifidelity: 0.0317 - val_loss: 0.0308 - val_antifidelity: 0.0308\n",
      "Epoch 71/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0303 - antifidelity: 0.0303 - val_loss: 0.0294 - val_antifidelity: 0.0294\n",
      "Epoch 72/200\n",
      "900/900 [==============================] - 17s 19ms/sample - loss: 0.0291 - antifidelity: 0.0291 - val_loss: 0.0282 - val_antifidelity: 0.0282\n",
      "Epoch 73/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0278 - antifidelity: 0.0278 - val_loss: 0.0270 - val_antifidelity: 0.0270\n",
      "Epoch 74/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0266 - antifidelity: 0.0266 - val_loss: 0.0258 - val_antifidelity: 0.0258\n",
      "Epoch 75/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0255 - antifidelity: 0.0255 - val_loss: 0.0247 - val_antifidelity: 0.0247\n",
      "Epoch 76/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0244 - antifidelity: 0.0244 - val_loss: 0.0236 - val_antifidelity: 0.0236\n",
      "Epoch 77/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0234 - antifidelity: 0.0234 - val_loss: 0.0227 - val_antifidelity: 0.0227\n",
      "Epoch 78/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0224 - antifidelity: 0.0224 - val_loss: 0.0217 - val_antifidelity: 0.0217\n",
      "Epoch 79/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0215 - antifidelity: 0.0215 - val_loss: 0.0208 - val_antifidelity: 0.0208\n",
      "Epoch 80/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0206 - antifidelity: 0.0206 - val_loss: 0.0199 - val_antifidelity: 0.0199\n",
      "Epoch 81/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0197 - antifidelity: 0.0197 - val_loss: 0.0191 - val_antifidelity: 0.0191\n",
      "Epoch 82/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0189 - antifidelity: 0.0189 - val_loss: 0.0183 - val_antifidelity: 0.0183\n",
      "Epoch 83/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0181 - antifidelity: 0.0181 - val_loss: 0.0175 - val_antifidelity: 0.0175\n",
      "Epoch 84/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0174 - antifidelity: 0.0174 - val_loss: 0.0168 - val_antifidelity: 0.0168\n",
      "Epoch 85/200\n",
      "900/900 [==============================] - 15s 17ms/sample - loss: 0.0166 - antifidelity: 0.0166 - val_loss: 0.0161 - val_antifidelity: 0.0161\n",
      "Epoch 86/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0160 - antifidelity: 0.0160 - val_loss: 0.0154 - val_antifidelity: 0.0154\n",
      "Epoch 87/200\n",
      "900/900 [==============================] - 15s 17ms/sample - loss: 0.0153 - antifidelity: 0.0153 - val_loss: 0.0148 - val_antifidelity: 0.0148\n",
      "Epoch 88/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0146 - antifidelity: 0.0146 - val_loss: 0.0142 - val_antifidelity: 0.0142\n",
      "Epoch 89/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0140 - antifidelity: 0.0140 - val_loss: 0.0136 - val_antifidelity: 0.0136\n",
      "Epoch 90/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0135 - antifidelity: 0.0135 - val_loss: 0.0130 - val_antifidelity: 0.0130\n",
      "Epoch 91/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0129 - antifidelity: 0.0129 - val_loss: 0.0125 - val_antifidelity: 0.0125\n",
      "Epoch 92/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0124 - antifidelity: 0.0124 - val_loss: 0.0120 - val_antifidelity: 0.0120\n",
      "Epoch 93/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0119 - antifidelity: 0.0119 - val_loss: 0.0115 - val_antifidelity: 0.0115\n",
      "Epoch 94/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0114 - antifidelity: 0.0114 - val_loss: 0.0110 - val_antifidelity: 0.0110\n",
      "Epoch 95/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0109 - antifidelity: 0.0109 - val_loss: 0.0105 - val_antifidelity: 0.0105\n",
      "Epoch 96/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0104 - antifidelity: 0.0104 - val_loss: 0.0101 - val_antifidelity: 0.0101\n",
      "Epoch 97/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0100 - antifidelity: 0.0100 - val_loss: 0.0097 - val_antifidelity: 0.0097\n",
      "Epoch 98/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0096 - antifidelity: 0.0096 - val_loss: 0.0093 - val_antifidelity: 0.0093\n",
      "Epoch 99/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0092 - antifidelity: 0.0092 - val_loss: 0.0089 - val_antifidelity: 0.0089\n",
      "Epoch 100/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0088 - antifidelity: 0.0088 - val_loss: 0.0085 - val_antifidelity: 0.0085\n",
      "Epoch 101/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0084 - antifidelity: 0.0084 - val_loss: 0.0082 - val_antifidelity: 0.0082\n",
      "Epoch 102/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0081 - antifidelity: 0.0081 - val_loss: 0.0078 - val_antifidelity: 0.0078\n",
      "Epoch 103/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0077 - antifidelity: 0.0077 - val_loss: 0.0075 - val_antifidelity: 0.0075\n",
      "Epoch 104/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0074 - antifidelity: 0.0074 - val_loss: 0.0072 - val_antifidelity: 0.0072\n",
      "Epoch 105/200\n",
      "900/900 [==============================] - 15s 17ms/sample - loss: 0.0071 - antifidelity: 0.0071 - val_loss: 0.0069 - val_antifidelity: 0.0069\n",
      "Epoch 106/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0068 - antifidelity: 0.0068 - val_loss: 0.0066 - val_antifidelity: 0.0066\n",
      "Epoch 107/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0065 - antifidelity: 0.0065 - val_loss: 0.0063 - val_antifidelity: 0.0063\n",
      "Epoch 108/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0063 - antifidelity: 0.0063 - val_loss: 0.0061 - val_antifidelity: 0.0061\n",
      "Epoch 109/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0060 - antifidelity: 0.0060 - val_loss: 0.0058 - val_antifidelity: 0.0058\n",
      "Epoch 110/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0057 - antifidelity: 0.0057 - val_loss: 0.0056 - val_antifidelity: 0.0056\n",
      "Epoch 111/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0055 - antifidelity: 0.0055 - val_loss: 0.0053 - val_antifidelity: 0.0053\n",
      "Epoch 112/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0053 - antifidelity: 0.0053 - val_loss: 0.0051 - val_antifidelity: 0.0051\n",
      "Epoch 113/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0050 - antifidelity: 0.0050 - val_loss: 0.0049 - val_antifidelity: 0.0049\n",
      "Epoch 114/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0048 - antifidelity: 0.0048 - val_loss: 0.0047 - val_antifidelity: 0.0047\n",
      "Epoch 115/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0046 - antifidelity: 0.0046 - val_loss: 0.0045 - val_antifidelity: 0.0045\n",
      "Epoch 116/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0044 - antifidelity: 0.0044 - val_loss: 0.0043 - val_antifidelity: 0.0043\n",
      "Epoch 117/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0042 - antifidelity: 0.0042 - val_loss: 0.0041 - val_antifidelity: 0.0041\n",
      "Epoch 118/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0040 - antifidelity: 0.0040 - val_loss: 0.0039 - val_antifidelity: 0.0039\n",
      "Epoch 119/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 0.0039 - antifidelity: 0.0039 - val_loss: 0.0038 - val_antifidelity: 0.0038\n",
      "Epoch 120/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0037 - antifidelity: 0.0037 - val_loss: 0.0036 - val_antifidelity: 0.0036\n",
      "Epoch 121/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0035 - antifidelity: 0.0035 - val_loss: 0.0035 - val_antifidelity: 0.0035\n",
      "Epoch 122/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0034 - antifidelity: 0.0034 - val_loss: 0.0033 - val_antifidelity: 0.0033\n",
      "Epoch 123/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0032 - antifidelity: 0.0032 - val_loss: 0.0032 - val_antifidelity: 0.0032\n",
      "Epoch 124/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0031 - antifidelity: 0.0031 - val_loss: 0.0030 - val_antifidelity: 0.0030\n",
      "Epoch 125/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0030 - antifidelity: 0.0030 - val_loss: 0.0029 - val_antifidelity: 0.0029\n",
      "Epoch 126/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 0.0028 - antifidelity: 0.0028 - val_loss: 0.0028 - val_antifidelity: 0.0028\n",
      "Epoch 127/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0027 - antifidelity: 0.0027 - val_loss: 0.0027 - val_antifidelity: 0.0027\n",
      "Epoch 128/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0026 - antifidelity: 0.0026 - val_loss: 0.0025 - val_antifidelity: 0.0025\n",
      "Epoch 129/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0025 - antifidelity: 0.0025 - val_loss: 0.0024 - val_antifidelity: 0.0024\n",
      "Epoch 130/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0024 - antifidelity: 0.0024 - val_loss: 0.0023 - val_antifidelity: 0.0023\n",
      "Epoch 131/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 0.0023 - antifidelity: 0.0023 - val_loss: 0.0022 - val_antifidelity: 0.0022\n",
      "Epoch 132/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0022 - antifidelity: 0.0022 - val_loss: 0.0021 - val_antifidelity: 0.0021\n",
      "Epoch 133/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0021 - antifidelity: 0.0021 - val_loss: 0.0020 - val_antifidelity: 0.0020\n",
      "Epoch 134/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0020 - antifidelity: 0.0020 - val_loss: 0.0019 - val_antifidelity: 0.0019\n",
      "Epoch 135/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0019 - antifidelity: 0.0019 - val_loss: 0.0019 - val_antifidelity: 0.0019\n",
      "Epoch 136/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0018 - antifidelity: 0.0018 - val_loss: 0.0018 - val_antifidelity: 0.0018\n",
      "Epoch 137/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0017 - antifidelity: 0.0017 - val_loss: 0.0017 - val_antifidelity: 0.0017\n",
      "Epoch 138/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0016 - antifidelity: 0.0016 - val_loss: 0.0016 - val_antifidelity: 0.0016\n",
      "Epoch 139/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0016 - antifidelity: 0.0016 - val_loss: 0.0015 - val_antifidelity: 0.0015\n",
      "Epoch 140/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0015 - antifidelity: 0.0015 - val_loss: 0.0015 - val_antifidelity: 0.0015\n",
      "Epoch 141/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0014 - antifidelity: 0.0014 - val_loss: 0.0014 - val_antifidelity: 0.0014\n",
      "Epoch 142/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 0.0014 - antifidelity: 0.0014 - val_loss: 0.0013 - val_antifidelity: 0.0013\n",
      "Epoch 143/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0013 - antifidelity: 0.0013 - val_loss: 0.0013 - val_antifidelity: 0.0013\n",
      "Epoch 144/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0012 - antifidelity: 0.0012 - val_loss: 0.0012 - val_antifidelity: 0.0012\n",
      "Epoch 145/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0012 - antifidelity: 0.0012 - val_loss: 0.0012 - val_antifidelity: 0.0012\n",
      "Epoch 146/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0011 - antifidelity: 0.0011 - val_loss: 0.0011 - val_antifidelity: 0.0011\n",
      "Epoch 147/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 0.0011 - antifidelity: 0.0011 - val_loss: 0.0011 - val_antifidelity: 0.0011\n",
      "Epoch 148/200\n",
      "896/900 [============================>.] - ETA: 0s - loss: 0.0010 - antifidelity: 0.0010\n",
      "Epoch 00148: ReduceLROnPlateau reducing learning rate to 5e-05.\n",
      "900/900 [==============================] - 133s 148ms/sample - loss: 0.0010 - antifidelity: 0.0010 - val_loss: 0.0010 - val_antifidelity: 0.0010\n",
      "Epoch 149/200\n",
      "900/900 [==============================] - 15s 16ms/sample - loss: 9.8158e-04 - antifidelity: 9.8158e-04 - val_loss: 9.7487e-04 - val_antifidelity: 9.7487e-04\n",
      "Epoch 150/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 9.5626e-04 - antifidelity: 9.5626e-04 - val_loss: 9.5239e-04 - val_antifidelity: 9.5239e-04\n",
      "Epoch 151/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 9.3254e-04 - antifidelity: 9.3254e-04 - val_loss: 9.2964e-04 - val_antifidelity: 9.2964e-04\n",
      "Epoch 152/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 9.1085e-04 - antifidelity: 9.1085e-04 - val_loss: 9.0710e-04 - val_antifidelity: 9.0710e-04\n",
      "Epoch 153/200\n",
      "896/900 [============================>.] - ETA: 0s - loss: 8.8912e-04 - antifidelity: 8.8912e-04\n",
      "Epoch 00153: ReduceLROnPlateau reducing learning rate to 2.5e-05.\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 8.8877e-04 - antifidelity: 8.8877e-04 - val_loss: 8.8433e-04 - val_antifidelity: 8.8433e-04\n",
      "Epoch 154/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 8.6670e-04 - antifidelity: 8.6670e-04 - val_loss: 8.7100e-04 - val_antifidelity: 8.7100e-04\n",
      "Epoch 155/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 8.5504e-04 - antifidelity: 8.5504e-04 - val_loss: 8.5850e-04 - val_antifidelity: 8.5850e-04\n",
      "Epoch 156/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 8.4349e-04 - antifidelity: 8.4349e-04 - val_loss: 8.4878e-04 - val_antifidelity: 8.4878e-04\n",
      "Epoch 157/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 8.3258e-04 - antifidelity: 8.3258e-04 - val_loss: 8.3688e-04 - val_antifidelity: 8.3688e-04\n",
      "Epoch 158/200\n",
      "896/900 [============================>.] - ETA: 0s - loss: 8.2178e-04 - antifidelity: 8.2178e-04\n",
      "Epoch 00158: ReduceLROnPlateau reducing learning rate to 1.25e-05.\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 8.2149e-04 - antifidelity: 8.2149e-04 - val_loss: 8.2592e-04 - val_antifidelity: 8.2592e-04\n",
      "Epoch 159/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 8.1089e-04 - antifidelity: 8.1089e-04 - val_loss: 8.1938e-04 - val_antifidelity: 8.1938e-04\n",
      "Epoch 160/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 8.0538e-04 - antifidelity: 8.0538e-04 - val_loss: 8.1488e-04 - val_antifidelity: 8.1488e-04\n",
      "Epoch 161/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.9999e-04 - antifidelity: 7.9999e-04 - val_loss: 8.0891e-04 - val_antifidelity: 8.0891e-04\n",
      "Epoch 162/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.9417e-04 - antifidelity: 7.9417e-04 - val_loss: 8.0191e-04 - val_antifidelity: 8.0191e-04\n",
      "Epoch 163/200\n",
      "896/900 [============================>.] - ETA: 0s - loss: 7.8926e-04 - antifidelity: 7.8926e-04\n",
      "Epoch 00163: ReduceLROnPlateau reducing learning rate to 6.25e-06.\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.8905e-04 - antifidelity: 7.8905e-04 - val_loss: 7.9647e-04 - val_antifidelity: 7.9647e-04\n",
      "Epoch 164/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.8349e-04 - antifidelity: 7.8349e-04 - val_loss: 7.9390e-04 - val_antifidelity: 7.9390e-04\n",
      "Epoch 165/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.8060e-04 - antifidelity: 7.8060e-04 - val_loss: 7.9102e-04 - val_antifidelity: 7.9102e-04\n",
      "Epoch 166/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.7763e-04 - antifidelity: 7.7763e-04 - val_loss: 7.8818e-04 - val_antifidelity: 7.8818e-04\n",
      "Epoch 167/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.7474e-04 - antifidelity: 7.7474e-04 - val_loss: 7.8521e-04 - val_antifidelity: 7.8521e-04\n",
      "Epoch 168/200\n",
      "896/900 [============================>.] - ETA: 0s - loss: 7.7186e-04 - antifidelity: 7.7186e-04\n",
      "Epoch 00168: ReduceLROnPlateau reducing learning rate to 3.125e-06.\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 7.7204e-04 - antifidelity: 7.7204e-04 - val_loss: 7.8293e-04 - val_antifidelity: 7.8293e-04\n",
      "Epoch 169/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 7.6910e-04 - antifidelity: 7.6910e-04 - val_loss: 7.8131e-04 - val_antifidelity: 7.8131e-04\n",
      "Epoch 170/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.6755e-04 - antifidelity: 7.6755e-04 - val_loss: 7.7985e-04 - val_antifidelity: 7.7985e-04\n",
      "Epoch 171/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.6610e-04 - antifidelity: 7.6610e-04 - val_loss: 7.7815e-04 - val_antifidelity: 7.7815e-04\n",
      "Epoch 172/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.6451e-04 - antifidelity: 7.6451e-04 - val_loss: 7.7669e-04 - val_antifidelity: 7.7669e-04\n",
      "Epoch 173/200\n",
      "896/900 [============================>.] - ETA: 0s - loss: 7.6297e-04 - antifidelity: 7.6297e-04\n",
      "Epoch 00173: ReduceLROnPlateau reducing learning rate to 1.5625e-06.\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.6302e-04 - antifidelity: 7.6302e-04 - val_loss: 7.7505e-04 - val_antifidelity: 7.7505e-04\n",
      "Epoch 174/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.6152e-04 - antifidelity: 7.6152e-04 - val_loss: 7.7399e-04 - val_antifidelity: 7.7399e-04\n",
      "Epoch 175/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.6073e-04 - antifidelity: 7.6073e-04 - val_loss: 7.7316e-04 - val_antifidelity: 7.7316e-04\n",
      "Epoch 176/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5995e-04 - antifidelity: 7.5995e-04 - val_loss: 7.7237e-04 - val_antifidelity: 7.7237e-04\n",
      "Epoch 177/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5916e-04 - antifidelity: 7.5916e-04 - val_loss: 7.7142e-04 - val_antifidelity: 7.7142e-04\n",
      "Epoch 178/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5837e-04 - antifidelity: 7.5837e-04 - val_loss: 7.7067e-04 - val_antifidelity: 7.7067e-04\n",
      "Epoch 179/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5753e-04 - antifidelity: 7.5753e-04 - val_loss: 7.6973e-04 - val_antifidelity: 7.6973e-04\n",
      "Epoch 180/200\n",
      "896/900 [============================>.] - ETA: 0s - loss: 7.5684e-04 - antifidelity: 7.5684e-04\n",
      "Epoch 00180: ReduceLROnPlateau reducing learning rate to 1e-06.\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5669e-04 - antifidelity: 7.5669e-04 - val_loss: 7.6897e-04 - val_antifidelity: 7.6897e-04\n",
      "Epoch 181/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5588e-04 - antifidelity: 7.5588e-04 - val_loss: 7.6854e-04 - val_antifidelity: 7.6854e-04\n",
      "Epoch 182/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 7.5532e-04 - antifidelity: 7.5532e-04 - val_loss: 7.6792e-04 - val_antifidelity: 7.6792e-04\n",
      "Epoch 183/200\n",
      "900/900 [==============================] - 14s 15ms/sample - loss: 7.5479e-04 - antifidelity: 7.5479e-04 - val_loss: 7.6727e-04 - val_antifidelity: 7.6727e-04\n",
      "Epoch 184/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5423e-04 - antifidelity: 7.5423e-04 - val_loss: 7.6678e-04 - val_antifidelity: 7.6678e-04\n",
      "Epoch 185/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5367e-04 - antifidelity: 7.5367e-04 - val_loss: 7.6615e-04 - val_antifidelity: 7.6615e-04\n",
      "Epoch 186/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5310e-04 - antifidelity: 7.5310e-04 - val_loss: 7.6579e-04 - val_antifidelity: 7.6579e-04\n",
      "Epoch 187/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5250e-04 - antifidelity: 7.5250e-04 - val_loss: 7.6524e-04 - val_antifidelity: 7.6524e-04\n",
      "Epoch 188/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5196e-04 - antifidelity: 7.5196e-04 - val_loss: 7.6466e-04 - val_antifidelity: 7.6466e-04\n",
      "Epoch 189/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5135e-04 - antifidelity: 7.5135e-04 - val_loss: 7.6417e-04 - val_antifidelity: 7.6417e-04\n",
      "Epoch 190/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5073e-04 - antifidelity: 7.5073e-04 - val_loss: 7.6356e-04 - val_antifidelity: 7.6356e-04\n",
      "Epoch 191/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.5014e-04 - antifidelity: 7.5014e-04 - val_loss: 7.6285e-04 - val_antifidelity: 7.6285e-04\n",
      "Epoch 192/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.4954e-04 - antifidelity: 7.4954e-04 - val_loss: 7.6241e-04 - val_antifidelity: 7.6241e-04\n",
      "Epoch 193/200\n",
      "900/900 [==============================] - 14s 16ms/sample - loss: 7.4893e-04 - antifidelity: 7.4893e-04 - val_loss: 7.6184e-04 - val_antifidelity: 7.6184e-04\n",
      "Epoch 194/200\n",
      "900/900 [==============================] - 13s 15ms/sample - loss: 7.4830e-04 - antifidelity: 7.4830e-04 - val_loss: 7.6107e-04 - val_antifidelity: 7.6107e-04\n",
      "Epoch 195/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 7.4767e-04 - antifidelity: 7.4767e-04 - val_loss: 7.6054e-04 - val_antifidelity: 7.6054e-04\n",
      "Epoch 196/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 7.4703e-04 - antifidelity: 7.4703e-04 - val_loss: 7.5974e-04 - val_antifidelity: 7.5974e-04\n",
      "Epoch 197/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 7.4639e-04 - antifidelity: 7.4639e-04 - val_loss: 7.5899e-04 - val_antifidelity: 7.5899e-04\n",
      "Epoch 198/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 7.4575e-04 - antifidelity: 7.4575e-04 - val_loss: 7.5837e-04 - val_antifidelity: 7.5837e-04\n",
      "Epoch 199/200\n",
      "900/900 [==============================] - 12s 14ms/sample - loss: 7.4506e-04 - antifidelity: 7.4506e-04 - val_loss: 7.5766e-04 - val_antifidelity: 7.5766e-04\n",
      "Epoch 200/200\n",
      "900/900 [==============================] - 13s 14ms/sample - loss: 7.4442e-04 - antifidelity: 7.4442e-04 - val_loss: 7.5710e-04 - val_antifidelity: 7.5710e-04\n",
      "Writing data to ./operator_history_5_qubits_2019.09.26.17.07.49.h5...\n"
     ]
    }
   ],
   "source": [
    "model = QPGA(N, 200).as_sequential()\n",
    "model.build(in_data.shape)\n",
    "model.compile(optimizer = Adam(lr = 0.0001),\n",
    "              loss = antifidelity,\n",
    "              metrics = [antifidelity])\n",
    "\n",
    "operator_vis = OperatorHistoryCallback(num_qubits=N, in_data=in_data, out_data=out_data)\n",
    "reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',\n",
    "                                              factor = 0.5,\n",
    "                                              cooldown = 4,\n",
    "                                              patience = 2,\n",
    "                                              verbose = 1,\n",
    "                                              min_lr = 1e-6)\n",
    "callbacks = [operator_vis, reduce_lr]\n",
    "\n",
    "history = model.fit(in_data, out_data,\n",
    "                    epochs = 200,\n",
    "                    validation_split = 0.1,\n",
    "                    callbacks = callbacks,\n",
    "                    verbose=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_frames(filepath, num_qubits=N, num_epochs=None):\n",
    "    \n",
    "    f = h5py.File(filepath, 'r')\n",
    "    \n",
    "    fidelities_train = np.array(f['fidelities_train'])\n",
    "    fidelities_val = np.array(f['fidelities_val'])\n",
    "    operators = np.array(f['operators'])\n",
    "    \n",
    "    num_epochs = len(fidelities_train) if num_epochs is None else num_epochs\n",
    "\n",
    "    operator_targ = extract_operator_from_circuit(QFT, num_qubits)\n",
    "    kets, bras = computational_basis_labels(num_qubits, include_bras=True)\n",
    "    \n",
    "    for i in pbar(range(num_epochs)):\n",
    "        \n",
    "        global_phase = np.mean(np.angle(operators[i]/operator_targ))\n",
    "        operator = operators[i] / np.exp(1j * global_phase)\n",
    "\n",
    "        # Make figure and axis layout\n",
    "        plt.rc('text', usetex=True)\n",
    "        plt.rc('font', family='serif')\n",
    "        fig = plt.figure(figsize=(12,9), tight_layout=True)\n",
    "        gs = mpl.gridspec.GridSpec(2, 2, height_ratios = [2, 1])\n",
    "\n",
    "        ax1 = fig.add_subplot(gs[0, 0])\n",
    "        ax2 = fig.add_subplot(gs[0, 1])\n",
    "\n",
    "        # Plot operator visualizations\n",
    "        hinton(operator, xlabels=kets, ylabels=bras, fig=fig, ax=ax1, title=\"$\\\\tilde{U}$\")\n",
    "        hinton(operator_targ, xlabels=kets, ylabels=bras, fig=fig, ax=ax2, title=\"$\\hat{U}$\")\n",
    "\n",
    "        # Plot antifidelity\n",
    "        fidel_val = fidelities_val[0:i]\n",
    "        fidel_train = fidelities_train[0:i]\n",
    "        ax_bot = fig.add_subplot(gs[1, :])\n",
    "        loss_plot(fidel_val, fidel_train, x_units='iterations', x_max = num_epochs, fig=fig, ax=ax_bot)\n",
    "\n",
    "        # Save frame\n",
    "        title = str(i).zfill(4)\n",
    "        plt.savefig(\"assets/frames/qft_{}.png\".format(title), dpi=300, bbox_inches='tight', pad_inches=0)\n",
    "        plt.close()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "2cf3d9c282c6432ca15ca3c9a5aadcc5",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "HBox(children=(IntProgress(value=0, max=151), HTML(value='')))"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/Users/ben/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:2369: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.\n",
      "  warnings.warn(\"This figure includes Axes that are not compatible \"\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n"
     ]
    }
   ],
   "source": [
    "plot_frames('./logs/operator_history_5_qubits_2019.09.26.17.07.49.h5', num_qubits=5, num_epochs=151)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define the nic_Edge cyclic colormap \n",
    "\n",
    "nic_edge_colors = np.array([\n",
    "    [0.191208, 0.191208, 0.191208],\n",
    "    [0.239484, 0.00545035, 0.614821],\n",
    "    [0.220593, 0.0617459, 0.863547],\n",
    "    [0.17509, 0.278988, 0.97794],\n",
    "    [0.143526, 0.576069, 0.998553],\n",
    "    [0.166456, 0.871883, 0.96594],\n",
    "    [0.376202, 0.993555, 0.981833],\n",
    "    [0.681996, 0.991297, 0.999239],\n",
    "    [0.954172, 0.952734, 0.94374],\n",
    "    [0.999735, 0.99301, 0.662896],\n",
    "    [0.979399, 0.991466, 0.357973],\n",
    "    [0.968771, 0.854967, 0.162659],\n",
    "    [0.999245, 0.556697, 0.144323],\n",
    "    [0.973959, 0.26223, 0.177946],\n",
    "    [0.852358, 0.0526707, 0.222974],\n",
    "    [0.593889, 0.00912724, 0.238855],\n",
    "    [0.191208, 0.191208, 0.191208]\n",
    "])\n",
    "nic_edge = ListedColormap(nic_edge_colors)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 864x432 with 7 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 429,
       "width": 851
      },
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "def plot_qft_figure(filepath, t1 = 0, t2 = 25, tmax = 50, figscale = 12, savefig = False):\n",
    "\n",
    "    f = h5py.File(filepath, 'r')\n",
    "\n",
    "    fidelities_train = np.array(f['fidelities_train'])\n",
    "    fidelities_val = np.array(f['fidelities_val'])\n",
    "    operators = np.array(f['operators'])\n",
    "    fidelity_init = 1 - np.array(f['fidelity_initial'])[-1]\n",
    "    operator_init = np.array(f['operator_initial'])\n",
    "    \n",
    "    fidelities_val = np.insert(fidelities_val, 0, fidelity_init, axis=0)\n",
    "    fidelities_train = np.insert(fidelities_train, 0, fidelity_init, axis=0)\n",
    "    operators = np.insert(operators, 0, operator_init, axis=0)\n",
    "    \n",
    "    operator_targ = extract_operator_from_circuit(QFT, N)\n",
    "    kets, bras = computational_basis_labels(N, include_bras=True)\n",
    "\n",
    "    global_phase1 = np.mean(np.angle(operators[t1]/operator_targ))\n",
    "    operator1 = operators[t1] / np.exp(1j * global_phase1)\n",
    "\n",
    "    global_phase2 = np.mean(np.angle(operators[t2]/operator_targ))\n",
    "    operator2 = operators[t2] / np.exp(1j * global_phase2)\n",
    "    \n",
    "    global_phase3 = np.mean(np.angle(operators[tmax]/operator_targ))\n",
    "    operator3 = operators[tmax] / np.exp(1j * global_phase3)\n",
    "\n",
    "    # Make figure and axis layout\n",
    "    plt.rc('text', usetex=True)\n",
    "    plt.rc('font', family='serif')\n",
    "    fig = plt.figure(figsize=(figscale, figscale*(1/3 + 1/6)), tight_layout=True)\n",
    "    gs = mpl.gridspec.GridSpec(2, 3, height_ratios = [2, 1])\n",
    "\n",
    "    ax1 = fig.add_subplot(gs[0, 0])\n",
    "    ax2 = fig.add_subplot(gs[0, 1])\n",
    "    ax3 = fig.add_subplot(gs[0, 2])\n",
    "    \n",
    "    # Plot antifidelity\n",
    "    fidel_val = fidelities_val[0:tmax+1]\n",
    "    fidel_train = fidelities_train[0:tmax+1]\n",
    "    ax_bot = fig.add_subplot(gs[1, :])\n",
    "    loss_plot(fidel_val, fidel_train, x_units='epochs', x_max = tmax+1, fig=fig, ax=ax_bot, log_fidelity=False)\n",
    "\n",
    "    # Plot operator visualizations\n",
    "    fidel1 = fidel_val[t1]\n",
    "    hinton(operator1, xlabels=kets, ylabels=bras, fig=fig, ax=ax1, title=\"$\\\\tilde{U}_{\"+str(t1)+\"}$\")\n",
    "    \n",
    "    fidel2 = fidel_val[t2]\n",
    "    hinton(operator2, xlabels=kets, ylabels=bras, fig=fig, ax=ax2, title=\"$\\\\tilde{U}_{\"+str(t2)+\"}$\")\n",
    "    \n",
    "    fidel_3 = fidel_val[tmax]\n",
    "    hinton(operator3, xlabels=kets, ylabels=bras, fig=fig, ax=ax3, title=\"$\\\\tilde{U}_{\"+str(tmax)+\"}$\")\n",
    "\n",
    "    if savefig:\n",
    "        plt.savefig(\"assets/qft_3panel.pdf\", dpi=300, bbox_inches='tight', pad_inches=0)\n",
    "    else:\n",
    "        plt.show()\n",
    "    \n",
    "plot_qft_figure('logs/operator_history_4_qubits_2019.08.27.11.03.16.h5', t1=10, t2=20, tmax=50, savefig=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "is_executing": false,
     "name": "#%%\n"
    }
   },
   "outputs": [
    {
     "ename": "NameError",
     "evalue": "name 'prepare_training_data' is not defined",
     "output_type": "error",
     "traceback": [
      "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[0;31mNameError\u001b[0m                                 Traceback (most recent call last)",
      "\u001b[0;32m<ipython-input-1-b37e5c4b8d00>\u001b[0m in \u001b[0;36m<module>\u001b[0;34m\u001b[0m\n\u001b[0;32m----> 1\u001b[0;31m \u001b[0min_data\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mout_data\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mprepare_training_data\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mcluster_state_generator\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;36m9\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;36m10\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m",
      "\u001b[0;31mNameError\u001b[0m: name 'prepare_training_data' is not defined"
     ]
    }
   ],
   "source": [
    "in_data, out_data = prepare_training_data(cluster_state_generator, 9, 10)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "raw",
   "metadata": {},
   "source": [
    "N=9\n",
    "model = QPGA(N, 100).as_sequential()\n",
    "model.compile(optimizer=Adam(lr=0.01), \n",
    "              loss=antifidelity, \n",
    "              metrics=[antifidelity])\n",
    "\n",
    "callback = StatePreparationHistoryCallback(num_qubits=N, input_state = in_data[0:1], target_state = out_data[0:1], filename=\"9_qubit_cluster_state.h5\")\n",
    "reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(monitor = 'loss', \n",
    "                                                       factor = 0.25,\n",
    "                                                       patience = 2,\n",
    "                                                       verbose=1,\n",
    "                                                       min_lr=1e-6)\n",
    "\n",
    "history = model.fit(in_data, out_data, epochs=101, \n",
    "                    callbacks=[callback, reduce_lr_callback], verbose = 2)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## MZI self-interference"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 50,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gaussian(omegas, sigma, omega0 = 0.0):\n",
    "    return 1/(sigma * np.sqrt(2 * np.pi)) * np.exp(-(omegas - omega0)**2 / (2 * sigma**2))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 51,
   "metadata": {},
   "outputs": [],
   "source": [
    "H = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]], dtype = np.complex128)\n",
    "BS = 1 / np.sqrt(2) * np.array([[1 + 0j, 0 + 1j], [0 + 1j, 1 + 0j]], dtype = np.complex128)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 52,
   "metadata": {},
   "outputs": [],
   "source": [
    "class MZI_single_photon:\n",
    "    def __init__(self, tau):\n",
    "        self.tau=tau\n",
    "    \n",
    "    def call(self, x):\n",
    "        a1, a2 = x\n",
    "        \n",
    "        # Hadamard op\n",
    "        a1, a2 = np.dot(H, np.array([a1, a2]))\n",
    "        \n",
    "        # Phase shift in frequency space\n",
    "        a1 = a1 * np.exp(1j * omegas * self.tau)\n",
    "        \n",
    "        # Hadamard op\n",
    "        a1, a2 = np.dot(H, np.array([a1, a2]))\n",
    "        \n",
    "        # Phase shift in frequency space\n",
    "        a1 = a1 * np.exp(1j * omegas * self.tau)\n",
    "        \n",
    "        return np.array([a1, a2])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 53,
   "metadata": {},
   "outputs": [],
   "source": [
    "n_omega = 999\n",
    "omega0 = 1e12\n",
    "sigma = 1e10\n",
    "omegas = np.linspace(omega0-5*sigma, omega0+5*sigma, n_omega)\n",
    "phi = np.reshape(gaussian(omegas, sigma, omega0=omega0), (n_omega, 1))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 54,
   "metadata": {},
   "outputs": [],
   "source": [
    "mzi = MZI_single_photon(np.pi/omega0)\n",
    "psi_in = np.transpose(phi * (np.array([1, -1]) / np.sqrt(2)))\n",
    "psi_out = mzi.call(psi_in)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 55,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 864x576 with 1 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 488,
       "width": 713
      },
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# fix, [[ax1, ax2], [ax3, ax4]] = plt.subplots(2, 2, figsize=(12,8))\n",
    "plt.figure(figsize=(12,8))\n",
    "plt.plot(omegas, np.real(psi_out[0]), label=\"$\\Re (\\phi(\\omega) a^\\dagger(\\omega))$\")\n",
    "plt.plot(omegas, np.imag(psi_out[0]), label=\"$\\Im (\\phi(\\omega) a^\\dagger(\\omega))$\")\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([9.404e-29+5.420e-28j, 1.469e-14-4.628e-13j, 9.580e-14-1.507e-12j, 3.476e-13-3.640e-12j, 9.857e-13-7.724e-12j, 2.430e-12-1.519e-11j, 5.463e-12-2.834e-11j, 1.148e-11-5.083e-11j, 2.290e-11-8.826e-11j, 4.378e-11-1.491e-10j, 8.078e-11-2.459e-10j, 1.445e-10-3.969e-10j, 2.514e-10-6.280e-10j,\n",
       "       4.267e-10-9.751e-10j, 7.081e-10-1.488e-09j, 1.150e-09-2.231e-09j, 1.832e-09-3.293e-09j, 2.864e-09-4.783e-09j, 4.397e-09-6.842e-09j, 6.637e-09-9.640e-09j, 9.854e-09-1.338e-08j, 1.440e-08-1.831e-08j, 2.072e-08-2.469e-08j, 2.936e-08-3.282e-08j, 4.101e-08-4.301e-08j, 5.645e-08-5.556e-08j,\n",
       "       7.661e-08-7.076e-08j, 1.025e-07-8.884e-08j, 1.353e-07-1.100e-07j, 1.763e-07-1.342e-07j, 2.265e-07-1.613e-07j, 2.873e-07-1.911e-07j, 3.595e-07-2.231e-07j, 4.442e-07-2.565e-07j, 5.418e-07-2.903e-07j, 6.524e-07-3.233e-07j, 7.756e-07-3.542e-07j, 9.106e-07-3.814e-07j, 1.056e-06-4.033e-07j,\n",
       "       1.209e-06-4.183e-07j, 1.367e-06-4.250e-07j, 1.526e-06-4.220e-07j, 1.684e-06-4.084e-07j, 1.834e-06-3.838e-07j, 1.974e-06-3.481e-07j, 2.099e-06-3.018e-07j, 2.204e-06-2.458e-07j, 2.286e-06-1.818e-07j, 2.343e-06-1.116e-07j, 2.372e-06-3.763e-08j, 2.372e-06+3.763e-08j, 2.343e-06+1.116e-07j,\n",
       "       2.286e-06+1.818e-07j, 2.204e-06+2.458e-07j, 2.099e-06+3.018e-07j, 1.974e-06+3.481e-07j, 1.834e-06+3.838e-07j, 1.684e-06+4.084e-07j, 1.526e-06+4.220e-07j, 1.367e-06+4.250e-07j, 1.209e-06+4.183e-07j, 1.056e-06+4.033e-07j, 9.106e-07+3.814e-07j, 7.756e-07+3.542e-07j, 6.524e-07+3.233e-07j,\n",
       "       5.418e-07+2.903e-07j, 4.442e-07+2.565e-07j, 3.595e-07+2.231e-07j, 2.873e-07+1.911e-07j, 2.265e-07+1.613e-07j, 1.763e-07+1.342e-07j, 1.353e-07+1.100e-07j, 1.025e-07+8.884e-08j, 7.661e-08+7.076e-08j, 5.645e-08+5.556e-08j, 4.101e-08+4.301e-08j, 2.936e-08+3.282e-08j, 2.072e-08+2.469e-08j,\n",
       "       1.440e-08+1.831e-08j, 9.854e-09+1.338e-08j, 6.637e-09+9.640e-09j, 4.397e-09+6.842e-09j, 2.864e-09+4.783e-09j, 1.832e-09+3.293e-09j, 1.150e-09+2.231e-09j, 7.081e-10+1.488e-09j, 4.267e-10+9.751e-10j, 2.514e-10+6.280e-10j, 1.445e-10+3.969e-10j, 8.078e-11+2.459e-10j, 4.378e-11+1.491e-10j,\n",
       "       2.290e-11+8.826e-11j, 1.148e-11+5.083e-11j, 5.463e-12+2.834e-11j, 2.430e-12+1.519e-11j, 9.857e-13+7.724e-12j, 3.476e-13+3.640e-12j, 9.580e-14+1.507e-12j, 1.469e-14+4.628e-13j, 9.404e-29+1.626e-27j])"
      ]
     },
     "execution_count": 21,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "psi_out[1]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## MZI fidelity"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 57,
   "metadata": {},
   "outputs": [],
   "source": [
    "# copied from mathematica output\n",
    "mins, means, maxes, dTlist = [[0.9999999997814122,0.9999999997657385,0.9999999997553899,0.9999999997440789,0.9999999996725557,0.9999999996821307,0.9999999995840114,0.999999999531819,0.9999999994834645,0.9999999994085402,0.9999999993643298,0.9999999992469342,0.9999999992363611,0.9999999990525668,0.9999999990028035,0.999999998699806,0.9999999985743435,0.9999999983734307,0.9999999983861594,0.9999999982773329,0.9999999976340339,0.999999997800278,0.9999999973569691,0.9999999971186628,0.9999999963740824,0.9999999964489161,0.9999999956128693,0.9999999951307086,0.9999999945163864,0.9999999932417833,0.9999999926317957,0.9999999919379414,0.9999999912571615,0.9999999895104512,0.9999999885816111,0.9999999892578981,0.999999985707281,0.9999999854117387,0.999999981732348,0.9999999822557745,0.9999999787102009,0.9999999733206566,0.9999999730739144,0.999999970525896,0.9999999641128942,0.999999961502413,0.9999999575957943,0.9999999503851502,0.9999999457468677,0.9999999356445995,0.9999999346400483,0.9999999186422192,0.9999999176487914,0.9999999043571527,0.9999999000848996,0.9999998790838079,0.9999998628350137,0.9999998436986733,0.9999998276401768,0.9999998108399631,0.9999997744110608,0.9999997367986011,0.9999997281390225,0.9999996685874545,0.999999699916612,0.9999995791914004,0.9999995724614772,0.9999994996433488,0.9999994706016898,0.9999994418716142,0.9999992369600751,0.9999992702544572,0.9999990145381895,0.9999989959319608,0.999998810121161,0.999998769697588,0.9999985487487967,0.9999983583489828,0.9999983596576333,0.9999980198823416,0.9999976256260611,0.9999975510437288,0.9999970587607798,0.9999967453430453,0.9999966492331176,0.9999962681985576,0.9999950782569401,0.9999952342147173,0.999994681109652,0.999993542003807,0.9999929627538948,0.9999917900819484,0.9999900187368918,0.9999897402559587,0.9999893649138224,0.9999873573284452,0.9999870536553608,0.9999843754485308,0.9999820026610575,0.9999811105867642,0.9999754681828317,0.999973968627077,0.9999702540900045,0.9999701108332112,0.9999655258227571,0.9999546559739171,0.9999518234192738,0.9999474758429733,0.9999412565406918,0.9999318124409996,0.9999212490860586,0.9999208660765434,0.999911678915575,0.9998969946774909,0.9998961433209793,0.9998752164159581,0.9998534863286004,0.999840689757182,0.9998141099086754,0.9997978261475381,0.999782380340272,0.9997629397622544,0.9997194399252433,0.9996822322380664,0.9996579282726215,0.9996192796232635,0.9995143807571485,0.9994813782257147,0.9994204980062091,0.9993623687669575,0.999259358590936,0.999204945756033,0.9990222486771199,0.9989544408007756,0.9988588925289957,0.9987453625633592,0.9986369704250689,0.9985091771717607,0.9982642138858097,0.9980664936823564,0.9974715856568698,0.9975090426570377,0.997207206693087,0.9969740128253015,0.9964689098977211,0.9959057173600588,0.9951694931662303,0.9947085818386616,0.9943121647307822,0.9934428633280936,0.9922938873332069,0.9921420675718045,0.9906845711644237,0.9896922220614027,0.9873124513252589,0.9864585469834167,0.9857804044487853,0.9823761394065517,0.9818238662824343,0.9792616453094946,0.9745395693592193,0.9762715128905718,0.9705717955344475,0.9655907423377705,0.96837737251582,0.9573137369767911,0.9593152752523421,0.9492406417901268,0.9411373455534228,0.9404131973749991,0.9346856219731933,0.9251002072651576,0.9085667489174075,0.9018798489674689,0.891986164625019,0.8787136496304688,0.8614977502650258,0.8553236313499928,0.82657434228339,0.793590567035433,0.8024552994233767,0.7562185271432691,0.772290734610493,0.710062643830522,0.680553509249187,0.6577404133871345,0.6239977690122896,0.6382785620225584,0.6071543499873473,0.49211339262127163,0.4799814518157115,0.468329198668031,0.3916744496034218,0.4079114582692841,0.34223824302667766,0.3032395256668304,0.22499679717940785,0.19941253117999147,0.16136998146745984,0.10694946053640386],[0.9999999999532118,0.9999999999424511,0.9999999999444793,0.9999999999437528,0.9999999999204485,0.9999999999283995,0.9999999999013889,0.9999999998927258,0.9999999998828912,0.9999999998579325,0.9999999998333664,0.9999999998211875,0.9999999998034717,0.9999999997738079,0.9999999997281204,0.9999999997070865,0.9999999996828511,0.9999999996340705,0.9999999995988612,0.9999999995424447,0.999999999459089,0.9999999994377232,0.9999999993608688,0.9999999992522471,0.9999999991775962,0.9999999990864779,0.9999999989858244,0.9999999988698206,0.9999999986665102,0.9999999985528839,0.9999999983329905,0.999999998145726,0.9999999979013258,0.9999999976350565,0.9999999974241971,0.9999999970356218,0.9999999968215562,0.9999999962375558,0.9999999958139818,0.999999995333977,0.9999999947806696,0.9999999943258172,0.9999999933335428,0.9999999926114401,0.9999999914345856,0.9999999905751451,0.9999999895159619,0.9999999881533702,0.9999999868264363,0.9999999853770348,0.9999999847823833,0.9999999817153874,0.9999999795221427,0.9999999763875618,0.9999999737936898,0.999999970368074,0.9999999667082589,0.9999999633617649,0.9999999585977682,0.9999999525307869,0.9999999469730169,0.9999999410014039,0.999999932078029,0.9999999238244275,0.9999999186516356,0.9999999063812766,0.9999998958632773,0.9999998804732106,0.9999998699920103,0.9999998538105301,0.9999998297093801,0.999999810575814,0.9999997949869016,0.9999997632737769,0.9999997379806959,0.9999996992516844,0.9999996525084959,0.9999996348539973,0.9999995654436814,0.9999995350516678,0.9999994553826188,0.9999993981394171,0.999999308026319,0.9999992388005806,0.99999918077091,0.9999990606308253,0.9999989057878597,0.9999988005686361,0.999998720138092,0.9999985468570725,0.999998284074182,0.9999980317535482,0.9999978474697812,0.9999975266325941,0.9999973072379994,0.9999970549598111,0.999996593259791,0.9999960178499523,0.9999956853885988,0.9999952553325974,0.9999945462290482,0.9999939267687253,0.9999929580233488,0.9999922857350896,0.999991384035834,0.9999903343083498,0.9999892947959204,0.99998790554328,0.9999864081354057,0.9999852002441001,0.9999823609714301,0.9999806826165208,0.9999780911454702,0.9999745386398083,0.9999726544949852,0.9999697611370534,0.9999656791944228,0.9999609102226382,0.9999565704933417,0.9999492342460068,0.9999475135125465,0.9999392359875553,0.9999321124426748,0.9999232564824443,0.9999136603497846,0.9999045961705485,0.9998863415317105,0.9998761841043956,0.9998635280286345,0.9998454857135896,0.9998270496069476,0.9998117313227431,0.9997778321558786,0.9997504609823958,0.9997230332102512,0.9996866169464171,0.9996632103230864,0.9996188022175035,0.9995626851657612,0.9995141882575017,0.9994491649194519,0.99938036783909,0.9993067177166436,0.9992008949276758,0.9991445468881044,0.9989984274827227,0.9989324541608698,0.9987153100634694,0.9985975686012368,0.9983638803767373,0.9982150178904988,0.9980428642022846,0.9977777543616941,0.9974597080383001,0.9972466143869524,0.9969225562662898,0.9965149588819869,0.9960729161458326,0.9954252377989106,0.9951849904731648,0.9946658590045252,0.9938714774175256,0.9929491438707078,0.9921347680371283,0.9912448952892651,0.9900974953625314,0.9892615073190753,0.9875959640232049,0.9864506679234019,0.9845166444517089,0.9822502673578278,0.9808544294953987,0.977340005629799,0.9739675158915406,0.9728120865781514,0.969386668062851,0.9669707735925896,0.9607571355311371,0.9565398012652963,0.952167576853597,0.9468954053286657,0.9397840993443265,0.9319488370972349,0.9229216221556508,0.9133013703521192,0.9059272165605077,0.8958301408164332,0.884167653734434,0.8770991313393894,0.8589549548762881,0.8416466615294068,0.8312862716813654,0.8165088324380476,0.7895122852803031,0.7682614025518674,0.7474585201909256,0.7279441746111058,0.7055872977938177,0.6775059298759342,0.6456391010521361],[1.0000000000047957,1.0000000000003326,1.0000000000054343,1.0000000000126956,0.99999999999612,1.0000000000121956,0.9999999999955025,1.0000000000008908,0.9999999999986326,1.0000000000015024,0.9999999999876095,1.0000000000010054,1.000000000005548,0.9999999999935911,0.99999999998581,0.9999999999967624,0.9999999999919349,0.9999999999757647,0.9999999999917726,0.9999999999959976,0.9999999999835121,0.9999999999700919,0.9999999999891098,0.9999999999406439,0.9999999999604126,0.9999999999412883,0.9999999999095057,0.9999999999776317,0.9999999999082494,0.999999999980639,0.9999999999712113,0.999999999907726,0.9999999999250782,0.9999999998364151,0.9999999998389741,0.9999999999178675,0.9999999999562379,0.9999999999351676,0.9999999997709454,0.9999999999158777,0.9999999998766862,0.9999999999124678,0.9999999995738342,0.9999999996890025,0.9999999996128521,0.9999999995309024,0.9999999995672844,0.9999999997509239,0.9999999992793542,0.9999999995523812,0.9999999993497997,0.9999999986636694,0.9999999992551021,0.9999999993047313,0.9999999986885084,0.999999998896568,0.9999999993909574,0.9999999981103076,0.9999999983792693,0.9999999975106126,0.9999999976470204,0.9999999970352649,0.9999999956510917,0.9999999987114516,0.9999999973373845,0.9999999936592592,0.9999999922924063,0.9999999927011727,0.9999999966942896,0.9999999878546499,0.9999999970446973,0.9999999932409844,0.9999999978228356,0.9999999871422507,0.9999999841429764,0.9999999908466242,0.999999987308767,0.9999999902719527,0.9999999660453892,0.9999999845369346,0.9999999853501742,0.9999999803093168,0.9999999590190203,0.9999999760535349,0.9999999764497062,0.9999999734723286,0.999999984693528,0.9999999198756919,0.9999999715036438,0.9999999527625586,0.9999999218304372,0.9999999738277554,0.9999998925258333,0.9999999196389208,0.9999998522154724,0.9999998466757128,0.9999998925248976,0.9999998824822337,0.9999998385401624,0.9999997504035929,0.9999997095783895,0.9999997039620171,0.999999780774874,0.9999997217055523,0.9999995831097835,0.9999997168401226,0.9999994979546578,0.9999995885438006,0.9999996381623834,0.999999564694051,0.999999320898185,0.9999992141383359,0.9999988692109011,0.9999986126513192,0.9999993184780716,0.9999996831740074,0.999999167461831,0.9999977545676693,0.9999980792945732,0.9999969859011916,0.9999968165019274,0.9999969934854778,0.9999952904059108,0.9999957489663379,0.9999974188689513,0.9999986753345157,0.9999936703124334,0.9999932085136994,0.9999954682161342,0.9999957011508147,0.9999946817976553,0.9999897065120139,0.9999944656508434,0.9999935282535026,0.9999880553087978,0.999993514307177,0.9999819974564592,0.9999735156649583,0.9999821286736839,0.9999825651390791,0.9999933654172971,0.9999933730076563,0.9999719997679825,0.9999774455603148,0.9999475861323109,0.9999577152486956,0.999932410805726,0.9999186930267648,0.9999254654879381,0.9999503095876167,0.999939709060743,0.9999385299375925,0.999946938722274,0.9998505361564838,0.9998727756255092,0.999955730429893,0.9999089658142292,0.9998249430169265,0.9999076870792226,0.9998798048959489,0.9998268713952878,0.9999043328556609,0.9997203937372717,0.9994904861538393,0.9993007542312786,0.9999237246156057,0.9997943239417191,0.9994746037662536,0.99945492240118,0.9997321375505799,0.9992059928811181,0.9991051593596436,0.9992611745998682,0.9992982302194414,0.9993873862768113,0.9983247100557165,0.9991607718313379,0.9989028605455896,0.9987356419642799,0.9984280640566987,0.9990505602865163,0.9992589687851093,0.9963627836324622,0.9972064540886579,0.9970982951381803,0.9951167613616827,0.9976092100073174,0.9941717200924015,0.9934066143184636,0.995741519987807,0.9958365629709232,0.997440845817588,0.9956360868520706,0.9889648014833534,0.9952043130951364,0.9852508769382735,0.9883263787360915,0.9867714968817615,0.9790467520551669,0.9822313076980173],[500000.,471893.9138887686,445367.731930522,420332.64428091585,396704.8332898746,374405.19287950074,353359.0636963746,333495.98315150576,314749.44951109437,297056.6992482514,280358.4969102729,264598.93679792184,249725.25579275697,235687.65670583598,222439.14155637924,209935.35422219528,198134.4319350739,186996.8651239397,176485.3651365325,166564.73939673352,157201.773579575,148365.12040944333,140025.19470918155,132154.07434870515,124725.40676151583,117714.32071612077,111097.3430469762,104852.32006616153,98958.34339267787,93395.67995103904,88145.7059047974,83190.84430380637,78514.50623646886,74101.03528994285,69935.65513236194,66004.42004157083,62294.1682147504,58792.477702607786,55487.62482060361,52368.54489797243,49424.79523312793,46646.520131423385,44024.41790821732,41549.70974676693,39214.11030668841,37009.79998457819,34929.398733926246,32965.94135666771,31112.854183651158,29363.933065947385,27713.322603315537,26155.49654028129,24685.239264195017,23297.628343323384,21988.018046513607,20752.023789252362,19585.507454046303,18484.56353597512,17445.506067033868,16464.85627548574,15539.330938910069,14665.831391950212,13841.433151960333,13063.376127816633,12329.05537911302,11636.012394802037,10981.926862082733,10364.608897976848,9781.991717585324,9232.124714477712,8713.166930048254,8223.380889973314,7761.12678713524,7324.856991536424,6913.110868823282,6524.509890072008,6157.75301646413,5811.612343399259,5484.9289894619205,5176.609216478308,4885.620767673251,4610.989411667166,4351.795680742583,4107.171792459715,3876.298744314732,3658.4035717136,3452.7567600811653,3258.6698024412135,3075.492894290252,2902.612758047451,2739.450589796972,2585.4601214483805,2440.1257918272167,2302.9610205725544,2173.506579062513,2051.3290529135975,1936.0193909062784,1827.191535478629,1724.48113020288,1627.5442999175298,1536.0564994308781,1449.7114269414376,1368.2199985373347,1291.3093803413374,1218.7220750611095,1150.2150598864584,1085.5589728472514,1024.537344907923,966.945875227615,912.5917471595212,861.2929826993928,812.8778332218966,767.1842044650058,724.0591138372665,683.358178231003,644.9451306266541,608.6913638698304,574.4755000936543,542.1829843448049,511.70570105272634,482.94161205793495,455.7944149875409,430.173220834225,405.99224965920047,383.1705434003727,361.6316948241767,341.3035917136193,322.1181754360685,304.01121308247104,286.92208241511963,270.79356890397355,255.57167417200827,241.20543520826845,227.64675374334738,214.850235216042,202.77303679204138,191.37472392581557,180.61713498547152,170.46425348734056,160.88208751253677,151.83855590177288,143.3033808474125,135.24798652315673,127.64540341197583,120.47017801197622,113.69828761789638,107.30705989292018,101.27509696153334,95.58220376928509,90.20932046960358,85.13845861129498,80.35264091308193,75.83584442354612,71.57294687617393,67.54967605990133,63.752562035650634,60.16889203887946,56.78666791715526,53.594565960256375,50.58189898831036,47.73858057104028,45.05509125832509,42.52244670901339,40.13216761128587,37.87625129385956,35.74714493298789,33.73772026555346,31.841249723592934,30.05138391035191,28.362130342459885,26.76783338705362,25.263155326678397,23.843058488572346,22.502788378502487,21.23785776268449,20.04403164449232,18.917313085659643,17.853929824502313,16.85032164635964,15.903128463970598,15.009179067877945,14.16548050919662,13.369208079199733,12.61769585217383,11.90842775988079,11.239029167743627,10.607258924553149,10.011001859077922,9.448261698456047,8.917154384659547,8.415901766654784,7.942825647140263,7.496342163930228,7.074956487172879,6.677257814649494,6.301914648398637,5.947670336851598,5.613338867554067,5.29780089638808,5.]]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 69,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 263,
       "width": 395
      },
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.figure(figsize=(6,4))\n",
    "plt.loglog()\n",
    "plt.plot(dTlist, 1 - np.array(maxes), label=\"Minimum\")\n",
    "plt.plot(dTlist, 1 - np.array(means), label=\"Mean\")\n",
    "plt.plot(dTlist, 1 - np.array(mins), label=\"Maximum\")\n",
    "plt.legend()\n",
    "plt.ylabel(\"Infidelity $1-\\mathcal{F}$\")\n",
    "plt.xlabel(\"Normalized pulse length $\\omega_0 \\cdot \\delta t$\")\n",
    "plt.savefig(\"assets/mzi_fidelities.pdf\", dpi=300, bbox_inches='tight', pad_inches=0)\n",
    "# plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.3"
  },
  "pycharm": {
   "stem_cell": {
    "cell_type": "raw",
    "metadata": {
     "collapsed": false
    },
    "source": []
   }
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


SyntaxError: invalid syntax (4056129653.py, line 6)